<a href="https://colab.research.google.com/github/racoope70/exploratory-daytrading/blob/main/deployable_multi_stock_ppo_model_selector_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "shimmy>=2.0.0"

In [ ]:
!pip -q install yfinance pywavelets transformers --upgrade

In [ ]:
!apt-get remove --purge -y cuda* libcuda* nvidia* || echo "No conflicting CUDA packages"
!apt-get autoremove -y
!apt-get clean

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'cuda-toolkit-12-4-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-7' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-8' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-5-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-2' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-3' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-4' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-5' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-6' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-8' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-9' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-13-0' fo

In [ ]:
!apt-get update -qq && apt-get install -y \
    libcusolver11 libcusparse11 libcurand10 libcufft10 libnppig10 libnppc10 libnppial10 \
    cuda-toolkit-12-4

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libnppig10
E: Unable to locate package libnppc10
E: Unable to locate package libnppial10


In [ ]:
!pip uninstall -y protobuf
!pip install protobuf==3.20.3


Found existing installation: protobuf 3.20.3
Uninstalling protobuf-3.20.3:
  Successfully uninstalled protobuf-3.20.3
  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
orbax-checkpoint 0.11.32 requires jax>=0.6.0, which is not installed.
dopamine-rl 4.1.2 requires jax>=0.1.72, which is not installed.
dopamine-rl 4.1.2 requires jaxlib>=0.1.51, which is not installed.
flax 0.11.2 requires jax>=0.6.0, which is not installed.
ydf 0.15.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
opentelemetry-proto 1.38.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
tf-k

In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com \
    cuml-cu12==25.2.0 cudf-cu12==25.2.0 cupy-cuda12x \
    dask-cuda==25.2.0 dask-cudf-cu12==25.2.0


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [ ]:
!pip install numba==0.60.0


In [ ]:
!pip install "stable-baselines3[extra]>=2.0.0" "gymnasium>=0.29" "shimmy>=2.0.0" \
  gym-anytrading yfinance pandas numpy scikit-learn xgboost joblib


In [ ]:
#!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance xgboost joblib
#!pip install matplotlib scikit-learn pandas

In [ ]:
!pip install tensorflow==2.18.0

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124


In [ ]:
!pip uninstall -y jax jaxlib

# Then restart the runtime/kernel, and import TF normally:
import tensorflow as tf

In [ ]:
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-12.4'
os.environ['PATH'] += ':/usr/local/cuda-12.4/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-12.4/lib64'


In [ ]:
# Step 7: authenticate with hugging face hub (optional)
# This allows for better access and avoids rate limits when downloading public models/datasets

# Authenticate with Hugging Face Hub
# Notebook_login()

In [ ]:
# --- Imports ---
import os, gc, time, json, pywt, logging, warnings
from datetime import datetime, timedelta, timezone
import hashlib

import numpy as np
import pandas as pd
import yfinance as yf

import sys
MANIFEST["versions"]["python"] = sys.version

# --- Config ---
USE_SENTIMENT = False
USE_REGIME    = True
TEST_MODE     = True

INTERVAL     = "1h"
PERIOD_DAYS  = 720
FWD_HORIZON  = 10
UP_THR       = 0.02
DN_THR       = -0.02

VAL_FRACTION = 0.20

LOCAL_OUT   = "multi_stock_feature_engineered_dataset.csv"
LOCAL_TRAIN = "train.csv"
LOCAL_VAL   = "val.csv"
PARQ_FULL   = "features_full.parquet"
PARQ_TRAIN  = "train.parquet"
PARQ_VAL    = "val.parquet"


# --- Paths (works in Colab or locally) ---
try:
    from google.colab import drive
    drive.mount("/content/drive")
    DRIVE_BASE = "/content/drive/MyDrive"
except Exception:
    DRIVE_BASE = os.getcwd()

DRIVE_DIR = os.path.join(DRIVE_BASE, "trading_data")
os.makedirs(DRIVE_DIR, exist_ok=True)


# --- Logging ---
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logging.info(f"yfinance version: {getattr(yf, '__version__', 'unknown')}")
logging.info(f"pandas version: {pd.__version__}")

MANIFEST_PATH = os.path.join(DRIVE_DIR, "artifact_manifest.json")
MANIFEST = {
    "created_utc": datetime.now(timezone.utc).isoformat(),
    "config": {
        "INTERVAL": INTERVAL,
        "PERIOD_DAYS": PERIOD_DAYS,
        "FWD_HORIZON": FWD_HORIZON,
        "UP_THR": UP_THR,
        "DN_THR": DN_THR,
        "VAL_FRACTION": VAL_FRACTION,
        "USE_REGIME": USE_REGIME,
        "USE_SENTIMENT": USE_SENTIMENT,
        "TEST_MODE": TEST_MODE,
    },
    "versions": {
        "python": None,
        "pandas": pd.__version__,
        "yfinance": getattr(yf, "__version__", "unknown"),
    },
}

def _sha256_file(path: str, chunk_size: int = 1024 * 1024) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        while True:
            b = f.read(chunk_size)
            if not b:
                break
            h.update(b)
    return h.hexdigest()

def _safe_json_dump(obj: dict, path: str):
    tmp = path + ".tmp"
    with open(tmp, "w") as f:
        json.dump(obj, f, indent=2, sort_keys=True)
    os.replace(tmp, path)

def manifest_record(key: str, path: str):
    MANIFEST.setdefault("artifacts", {})
    MANIFEST["artifacts"][key] = {
        "path": os.path.abspath(path),
        "sha256": _sha256_file(path) if os.path.exists(path) else None,
        "bytes": int(os.path.getsize(path)) if os.path.exists(path) else None,
        "written_utc": datetime.now(timezone.utc).isoformat(),
    }
    _safe_json_dump(MANIFEST, MANIFEST_PATH)

# write initial manifest once
_safe_json_dump(MANIFEST, MANIFEST_PATH)


# --- Universe ---
ticker_list = [
    "AAPL","TSLA","MSFT","GOOGL","AMZN","NVDA","META","BRK-B","JPM","JNJ",
    "XOM","V","PG","UNH","MA","HD","LLY","MRK","PEP","KO",
    "BAC","ABBV","AVGO","PFE","COST","CSCO","TMO","ABT","ACN","WMT",
    "MCD","ADBE","DHR","CRM","NKE","INTC","QCOM","NEE","AMD","TXN",
    "AMGN","UPS","LIN","PM","UNP","BMY","LOW","RTX","CVX","IBM",
    "GE","SBUX","ORCL",
]
SYMBOLS = ["AAPL", "NVDA", "MSFT"] if TEST_MODE else ticker_list


# --- Optional sentiment model ---
if USE_SENTIMENT:
    try:
        import torch
        from transformers import pipeline

        device_id = 0 if torch.cuda.is_available() else -1
        sentiment_pipeline = pipeline("sentiment-analysis", model="ProsusAI/finbert", device=device_id)
        logging.info("FinBERT sentiment enabled.")
    except Exception as e:
        logging.warning(f"Could not init FinBERT; disabling sentiment. Err: {e}")
        USE_SENTIMENT = False
        sentiment_pipeline = None
else:
    sentiment_pipeline = None

# --- Helpers ---
def _check_deploy_outputs(prefix: str, folder: str) -> dict:
    need = [
        "_model.zip",
        "_vecnorm.pkl",
        "_features.json",
        "_probability_config.json",
        "_model_info.json",
    ]
    return {s: os.path.exists(os.path.join(folder, f"{prefix}{s}")) for s in need}

def _check_run_outputs(prefix, run_folder):
    need = ["_predictions.csv","_predictions_compat.csv","_trades.csv"]
    return {s: os.path.exists(os.path.join(run_folder, prefix+s)) for s in need}

def _latest_manifest_in_run(run_folder):
    if not run_folder:
        return None
    p = os.path.join(run_folder, "artifact_manifest.json")
    return p if os.path.exists(p) else None

def _force_datetime_column(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure a tz-naive 'Datetime' column exists and data is sorted/deduped by time."""
    if isinstance(df.index, pd.DatetimeIndex):
        try:
            if df.index.tz is not None:
                df.index = df.index.tz_convert(None)
        except Exception:
            try:
                df.index = df.index.tz_localize(None)
            except Exception:
                pass
        df.index.name = "Datetime"
        df = df.reset_index()
    else:
        if not isinstance(df.index, pd.RangeIndex):
          df = df.reset_index()
        first = df.columns[0]
        if np.issubdtype(df[first].dtype, np.datetime64):
            df = df.rename(columns={first: "Datetime"})
        elif "Date" in df.columns:
            df["Datetime"] = pd.to_datetime(df["Date"])
        elif "Datetime" not in df.columns:
            df["Datetime"] = pd.to_datetime(df[first], errors="coerce")

    if "Datetime" not in df.columns:
        raise KeyError("Failed to construct 'Datetime' column from yfinance output.")

    df["Datetime"] = pd.to_datetime(df["Datetime"])
    df = (
        df.drop_duplicates(subset=["Datetime"])
          .sort_values("Datetime")
          .reset_index(drop=True)
    )
    return df


def _normalize_ohlcv(df_in: pd.DataFrame, ticker: str) -> pd.DataFrame:
    """Normalize yfinance OHLCV columns to: Open, High, Low, Close, Adj Close, Volume."""
    import re
    df = df_in.copy()

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            " ".join([str(p) for p in col if p is not None and str(p) != ""]).strip()
            for col in df.columns
        ]

    df.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]

    tkr = ticker.upper().replace("-", "[- ]?")
    cleaned = {}
    for c in df.columns:
        c_up = c.upper()
        c2 = re.sub(rf"^(?:{tkr})[\s/_-]+", "", c_up)   # leading ticker
        c2 = re.sub(rf"[\s/_-]+(?:{tkr})$", "", c2)     # trailing ticker
        cleaned[c] = c2.title()

    if any(cleaned[c] != c for c in df.columns):
        df = df.rename(columns=cleaned)

    cols_ci = {c.lower(): c for c in df.columns}
    wants = {
        "Open":      ["open"],
        "High":      ["high"],
        "Low":       ["low"],
        "Close":     ["close", "last"],
        "Adj Close": ["adj close", "adj_close", "adjclose", "adjusted close"],
        "Volume":    ["volume", "vol"],
    }

    rename_map = {}
    for desired, alts in wants.items():
        if desired.lower() in cols_ci:
            rename_map[cols_ci[desired.lower()]] = desired
            continue
        for a in alts:
            if a in cols_ci:
                rename_map[cols_ci[a]] = desired
                break

    if rename_map:
        df = df.rename(columns=rename_map)

    return df


def download_stock_data(ticker, interval="1h", period_days=720, max_retries=5, sleep_base=3):
    """Download intraday data via yfinance with normalization and a history() fallback."""
    period_str = f"{int(period_days)}d"

    def _postprocess(df: pd.DataFrame) -> pd.DataFrame:
        df = _normalize_ohlcv(df, ticker)
        df = _force_datetime_column(df)
        needed = {"Open", "High", "Low", "Close", "Volume"}
        missing = needed - set(df.columns)
        if missing:
            logging.debug(f"[{ticker}] columns received: {list(df.columns)}")
            raise ValueError(f"Missing OHLCV columns after normalize: {missing}")
        if "Adj Close" not in df.columns:
            df["Adj Close"] = df["Close"]
        return df

    for attempt in range(1, max_retries + 1):
        try:
            logging.info(f"[{ticker}] Attempt {attempt}: download(period={period_str}, interval={interval})")
            df = yf.download(
                tickers=ticker,
                period=period_str,
                interval=interval,
                progress=False,
                auto_adjust=False,
                group_by="column",
                threads=False,
                prepost=False,
                repair=True,
            )
            if df is None or df.empty:
                raise ValueError("Empty data from download()")
            if attempt == 1:
                logging.debug(f"[{ticker}] raw columns (download): {list(df.columns)}")

            df = _postprocess(df)
            df["Symbol"] = ticker
            logging.info(f"[{ticker}] rows: {len(df)} from {df['Datetime'].min()} to {df['Datetime'].max()}")
            return df

        except Exception as e1:
            logging.warning(f"[{ticker}] download error: {e1} | trying history() fallback")
            try:
                hist = yf.Ticker(ticker).history(
                    period=period_str,
                    interval=interval,
                    auto_adjust=False,
                    actions=False,
                )
                if hist is None or hist.empty:
                    raise ValueError("Empty data from history()")
                if attempt == 1:
                    logging.debug(f"[{ticker}] raw columns (history): {list(hist.columns)}")

                df = _postprocess(hist)
                df["Symbol"] = ticker
                logging.info(f"[{ticker}] (fallback) rows: {len(df)} from {df['Datetime'].min()} to {df['Datetime'].max()}")
                return df

            except Exception as e2:
                wait = sleep_base * attempt
                logging.warning(f"[{ticker}] history() error: {e2} | retrying in {wait}s")
                time.sleep(wait)

    logging.error(f"[{ticker}] Failed to download after {max_retries} attempts.")
    return None


def denoise_wavelet(series, wavelet="db1", level=2):
    s = pd.Series(series).astype(float).ffill().bfill().to_numpy()
    try:
        coeffs = pywt.wavedec(s, wavelet, mode="symmetric", level=level)
        for i in range(1, len(coeffs)):
            coeffs[i] = np.zeros_like(coeffs[i])
        rec = pywt.waverec(coeffs, wavelet, mode="symmetric")
        return pd.Series(rec[:len(s)], index=series.index)
    except Exception as e:
        logging.warning(f"Wavelet denoising failed ({e}); using raw Close.")
        return pd.Series(s, index=series.index)


def score_sentiment(texts):
    if not USE_SENTIMENT or sentiment_pipeline is None:
        return [0.0] * len(texts)
    try:
        outputs = sentiment_pipeline(texts, truncation=True, max_length=256, batch_size=32)
        scores = []
        for r in outputs:
            label = r["label"].lower()
            if label == "positive":
                scores.append(+float(r["score"]))
            elif label == "negative":
                scores.append(-float(r["score"]))
            else:
                scores.append(0.0)
        return scores
    except Exception as e:
        logging.error(f"Sentiment scoring error: {e}")
        return [0.0] * len(texts)


def add_regime(df: pd.DataFrame) -> pd.DataFrame:
    df["Vol20"] = df["Close"].pct_change().rolling(20).std()
    df["Ret20"] = df["Close"].pct_change(20)
    vol_hi   = (df["Vol20"] > df["Vol20"].median()).astype(int)
    trend_hi = (df["Ret20"].abs() > df["Ret20"].abs().median()).astype(int)
    df["Regime4"] = vol_hi * 2 + trend_hi
    return df


def compute_enhanced_features(df: pd.DataFrame) -> pd.DataFrame:
    # Always copy first so we never mutate the caller's df
    df = df.copy()

    # Ensure Symbol exists
    if "Symbol" not in df.columns:
        df["Symbol"] = "UNKNOWN"

    # Flatten/clean columns if yfinance produced MultiIndex or duplicates
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df.loc[:, ~df.columns.duplicated()]

    # ---- REQUIRED COLS CHECK (add this) ----
    req = {"Open", "High", "Low", "Close", "Volume", "Datetime", "Symbol"}
    missing = req - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    # ---- Indicators ----
    df["SMA_20"] = df["Close"].rolling(20).mean()
    df["STD_20"] = df["Close"].rolling(20).std()
    df["Upper_Band"] = df["SMA_20"] + 2 * df["STD_20"]
    df["Lower_Band"] = df["SMA_20"] - 2 * df["STD_20"]

    df["Lowest_Low"]   = df["Low"].rolling(14).min()
    df["Highest_High"] = df["High"].rolling(14).max()
    denom = (df["Highest_High"] - df["Lowest_Low"]).replace(0, np.nan)
    df["Stoch"] = ((df["Close"] - df["Lowest_Low"]) / denom) * 100

    df["ROC"] = df["Close"].pct_change(10)
    df["OBV"] = (np.sign(df["Close"].diff()).fillna(0) * df["Volume"].fillna(0)).cumsum()

    tp = (df["High"] + df["Low"] + df["Close"]) / 3
    sma_tp = tp.rolling(20).mean()
    md = (tp - sma_tp).abs().rolling(20).mean()
    df["CCI"] = (tp - sma_tp) / (0.015 * md)

    df["EMA_10"] = df["Close"].ewm(span=10, adjust=False).mean()
    df["EMA_50"] = df["Close"].ewm(span=50, adjust=False).mean()
    ema12 = df["Close"].ewm(span=12, adjust=False).mean()
    ema26 = df["Close"].ewm(span=26, adjust=False).mean()
    df["MACD_Line"]   = ema12 - ema26
    df["MACD_Signal"] = df["MACD_Line"].ewm(span=9, adjust=False).mean()

    delta = df["Close"].diff()
    gain  = delta.clip(lower=0).rolling(14).mean()
    loss  = (-delta.clip(upper=0)).rolling(14).mean()
    rs = gain / (loss.replace(0, np.nan))
    df["RSI"] = 100 - (100 / (1 + rs))

    tr = pd.concat(
        [
            (df["High"] - df["Low"]),
            (df["High"] - df["Close"].shift()).abs(),
            (df["Low"] - df["Close"].shift()).abs(),
        ],
        axis=1,
    ).max(axis=1)
    df["ATR"] = tr.rolling(14).mean()

    df["Volatility"] = df["Close"].pct_change().rolling(20).std()
    df["Denoised_Close"] = denoise_wavelet(df["Close"].ffill())

    if USE_REGIME:
        df = add_regime(df)

    if USE_SENTIMENT and len(df):
        headline = f"{df['Symbol'].iloc[0]} is expected to perform well in the market."
        try:
            score = score_sentiment([headline])[0]
        except Exception as e:
            logging.warning(f"Sentiment scoring failed for {df['Symbol'].iloc[0]}: {e}")
            score = 0.0
        df["SentimentScore"] = float(score)
    else:
        df["SentimentScore"] = 0.0

    df["Delta"] = df["Close"].pct_change(1).fillna(0)
    df["Gamma"] = df["Delta"].diff().fillna(0)

    # Drop NaNs created by rolling indicators
    df = df.dropna().reset_index(drop=True)

    # Keep Symbol last (optional)
    cols = [c for c in df.columns if c != "Symbol"] + ["Symbol"]
    return df[cols]


# --- Build dataset ---
# --- Build dataset ---
all_dfs = []

for i, ticker in enumerate(SYMBOLS, 1):
    logging.info(f"[{i}/{len(SYMBOLS)}] Processing {ticker}")
    raw = download_stock_data(
        ticker,
        interval=INTERVAL,
        period_days=PERIOD_DAYS,
        max_retries=5,
        sleep_base=3
    )

    if raw is None or raw.empty:
        logging.warning(f"No data for {ticker}")
        continue

    # Data provenance goes HERE (right after successful download)
    MANIFEST.setdefault("data_provenance", {})
    MANIFEST["data_provenance"][ticker] = {
        "rows": int(len(raw)),
        "start": str(raw["Datetime"].min()) if "Datetime" in raw.columns else None,
        "end": str(raw["Datetime"].max()) if "Datetime" in raw.columns else None,
    }
    _safe_json_dump(MANIFEST, MANIFEST_PATH)

    try:
        features = compute_enhanced_features(raw)
        if features is not None and not features.empty:
            logging.info(f"[{ticker}] feature rows: {len(features)}")
            all_dfs.append(features)
        else:
            logging.warning(f"Feature set empty for {ticker}")
    except Exception as e:
        logging.error(f"Feature engineering failed for {ticker}: {e}")
    finally:
        del raw
        try:
            del features
        except NameError:
            pass
        gc.collect()
        time.sleep(0.5)

if not all_dfs:
    logging.warning("No usable data found for any ticker.")
    raise SystemExit

final_df = pd.concat(all_dfs, ignore_index=True)
logging.info(f"Combined dataset shape: {final_df.shape}")

if "Repaired?" in final_df.columns:
    final_df = final_df.drop(columns=["Repaired?"])

dt = pd.to_datetime(final_df["Datetime"], errors="coerce")

# if tz-naive -> localize to UTC, else convert to UTC
if getattr(dt.dt, "tz", None) is None:
    dt = dt.dt.tz_localize("UTC")
else:
    dt = dt.dt.tz_convert("UTC")

final_df["Datetime"] = dt.dt.tz_convert("America/New_York")

dt = final_df["Datetime"]
rth_mask = (
    (dt.dt.weekday < 5) &
    (dt.dt.time >= pd.to_datetime("09:30").time()) &
    (dt.dt.time <  pd.to_datetime("16:00").time())
)
final_df = final_df[rth_mask].reset_index(drop=True)


def relabel(df):
    df = df.copy()
    df["Return"] = (df["Close"].shift(-FWD_HORIZON) - df["Close"]) / df["Close"]
    df["Target"] = np.select(
        [df["Return"] > UP_THR, df["Return"] < DN_THR],
        [1, -1],
        default=0,
    )
    return df


final_df = relabel(final_df)
final_df = final_df.sort_values(["Symbol", "Datetime"]).reset_index(drop=True)

final_df["__i"] = final_df.groupby("Symbol").cumcount()
final_df["__n"] = final_df.groupby("Symbol")["__i"].transform("max") + 1
final_df = final_df[final_df["__i"] < final_df["__n"] - FWD_HORIZON].copy()
final_df = final_df.drop(columns=["__i", "__n"]).reset_index(drop=True)

feature_cols = [c for c in final_df.columns if c not in ["Target", "Return", "Symbol", "Datetime"]]
MANIFEST["feature_manifest"] = feature_cols[:]  # ordered list
_safe_json_dump(MANIFEST, MANIFEST_PATH)

final_df = final_df.dropna(subset=feature_cols).reset_index(drop=True)

ORDER_LAST = ["Target", "Return", "Symbol"]
cols = [c for c in final_df.columns if c not in ORDER_LAST] + ORDER_LAST
final_df = final_df[cols]


def summarize(df):
    print("Combined dataset shape:", df.shape)
    print("Range:", df["Datetime"].min(), "-", df["Datetime"].max())
    print("Per-ticker counts:")
    print(df["Symbol"].value_counts().to_string())

    na_cols = df.columns[df.isna().any()]
    if len(na_cols):
        print("\nColumns with NaNs:")
        print(df[na_cols].isna().sum().sort_values(ascending=False).to_string())
    else:
        print("\nNo NaNs detected.")

    print("\nLabel counts:")
    print(df["Target"].value_counts().sort_index().to_string())
    print("\nLabel ratios (%):")
    print((df["Target"].value_counts(normalize=True) * 100).round(2).to_string())


summarize(final_df)
# --- Per-symbol time split (safer than global split) ---
train_parts, val_parts = [], []

for sym, g in final_df.groupby("Symbol", sort=False):
    g = g.sort_values("Datetime").reset_index(drop=True)
    cut = int((1.0 - VAL_FRACTION) * len(g))

    # optional safety: skip symbols that are too small to split
    if cut <= 0 or cut >= len(g):
        logging.warning(f"Skipping {sym} due to insufficient rows for split.")
        continue

    train_parts.append(g.iloc[:cut])
    val_parts.append(g.iloc[cut:])

train_df = pd.concat(train_parts, ignore_index=True)
val_df   = pd.concat(val_parts, ignore_index=True)

# ensure stable ordering
train_df = train_df.sort_values(["Symbol", "Datetime"]).reset_index(drop=True)
val_df   = val_df.sort_values(["Symbol", "Datetime"]).reset_index(drop=True)

print(f"\nTrain: {train_df.shape},  Val: {val_df.shape}")

# -----------------------------
# Save locally (CSV)
# -----------------------------
final_df.to_csv(LOCAL_OUT, index=False)
train_df.to_csv(LOCAL_TRAIN, index=False)
val_df.to_csv(LOCAL_VAL, index=False)

manifest_record("local_full_csv",  os.path.abspath(LOCAL_OUT))
manifest_record("local_train_csv", os.path.abspath(LOCAL_TRAIN))
manifest_record("local_val_csv",   os.path.abspath(LOCAL_VAL))

logging.info(f"Saved CSVs: {LOCAL_OUT}, {LOCAL_TRAIN}, {LOCAL_VAL}")

# Save locally (Parquet)
final_df.to_parquet(PARQ_FULL, index=False)
train_df.to_parquet(PARQ_TRAIN, index=False)
val_df.to_parquet(PARQ_VAL, index=False)

manifest_record("local_full_parquet",  os.path.abspath(PARQ_FULL))
manifest_record("local_train_parquet", os.path.abspath(PARQ_TRAIN))
manifest_record("local_val_parquet",   os.path.abspath(PARQ_VAL))

logging.info(f"Saved Parquets: {PARQ_FULL}, {PARQ_TRAIN}, {PARQ_VAL}")

# Save to Google Drive (CSV)
final_path = os.path.join(DRIVE_DIR, LOCAL_OUT)
train_path = os.path.join(DRIVE_DIR, LOCAL_TRAIN)
val_path   = os.path.join(DRIVE_DIR, LOCAL_VAL)

final_df.to_csv(final_path, index=False)
train_df.to_csv(train_path, index=False)
val_df.to_csv(val_path, index=False)

manifest_record("drive_full_csv",  final_path)
manifest_record("drive_train_csv", train_path)
manifest_record("drive_val_csv",   val_path)

logging.info(f"Saved to Google Drive:\n- {final_path}\n- {train_path}\n- {val_path}")

# cleanup
del all_dfs, final_df, train_df, val_df
gc.collect()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Combined dataset shape: (14883, 34)
Range: 2023-04-11 12:30:00-04:00 - 2026-02-12 12:30:00-05:00
Per-ticker counts:
Symbol
AAPL    4961
MSFT    4961
NVDA    4961

No NaNs detected.

Label counts:
Target
-1     1962
 0    10460
 1     2461

Label ratios (%):
Target
 0    70.28
 1    16.54
-1    13.18

Time split cutoff @ 2025-07-21 14:30:00-04:00
Train: (11904, 34),  Val: (2979, 34)


0

In [ ]:
df = pd.read_csv("multi_stock_feature_engineered_dataset.csv")
print(df.head())


                    Datetime   Adj Close       Close        High         Low  \
0  2023-04-11 12:30:00-04:00  159.107223  161.411697  161.520004  160.839996   
1  2023-04-11 12:30:00-04:00  277.271117  283.191589  283.839996  282.670013   
2  2023-04-11 12:30:00-04:00   27.287317   27.310001   27.327000   27.204998   
3  2023-04-11 13:30:00-04:00  159.401951  161.710693  161.830002  161.309998   
4  2023-04-11 13:30:00-04:00  277.798282  283.730011  283.769989  283.109985   

         Open   Volume      SMA_20    STD_20  Upper_Band  ...  Denoised_Close  \
0  161.119995  4141413  162.552909  1.583345  165.719598  ...      161.288048   
1  283.100006  2139616  286.746831  2.961618  292.670066  ...      283.160400   
2   27.278000  2455490   27.157355  0.281431   27.720216  ...       27.313876   
3  161.415604  4299346  162.475694  1.584958  165.645610  ...      161.288048   
4  283.204987  2247973  286.741832  2.966880  292.675591  ...      283.160400   

      Vol20     Ret20  Regime4  

In [ ]:
# PPO walkforward training + selector
import os, gc, time, json, logging, glob
import shutil
from threading import Lock
from datetime import datetime, timedelta
import heapq

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  # optional for ad-hoc plots

import torch
import gymnasium as gym
from gymnasium.spaces import Box as GBox

import yfinance as yf
from gym_anytrading.envs import StocksEnv

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.utils import set_random_seed

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="gymnasium")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="jupyter_client.session")
warnings.filterwarnings("ignore", message=".*Gym has been unmaintained.*")

import random
SEED = 42

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

    try:
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
        # Determinism (slower, but reproducible)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    except Exception:
        pass

    # SB3 helper
    try:
        set_random_seed(seed)
    except Exception:
        pass

seed_everything(SEED)

# ---- Sharpe annualization helper (intraday heuristic: 6.5 hrs * 252) ----
def _annualization_factor(_df_like=None) -> float:
    """Annualization factor for intraday bars (6.5 trading hours × 252 days)."""
    return np.sqrt(252 * 6.5)

FEATURES_FALLBACK_ACTIVE = False
try:
    from feature_engineering import compute_enhanced_features  # real pipeline
except Exception as e:
    FEATURES_FALLBACK_ACTIVE = True

    def compute_enhanced_features(df_in: pd.DataFrame) -> pd.DataFrame:
        df_out = df_in.copy()
        if "Datetime" in df_out.columns:
            df_out["Datetime"] = pd.to_datetime(df_out["Datetime"])
            df_out = df_out.sort_values("Datetime").reset_index(drop=True)
        if "Close" not in df_out.columns:
            raise ValueError("compute_enhanced_features: missing required column 'Close'")
        return df_out

    logging.warning(
        f"[WARN] compute_enhanced_features fallback is active (import failed: {e}). "
        "Training from your prebuilt CSV may still work, but LIVE_MODE inference "
        "won't match your real feature pipeline."
    )

# Flags
ENABLE_VOL_CONTROLS = True
VOL_LOOKBACK   = 20
VOL_EWM_SPAN   = 20
VOL_GATE_Q     = 0.95
VOL_SCALE_K    = 2.0
VOL_MIN_SCALE  = 0.25
TP_BASE = 0.030
SL_BASE = 0.020
TP_MIN, TP_MAX = 0.010, 0.080
SL_MIN, SL_MAX = 0.005, 0.060

# Flags
ENABLE_SENTIMENT = False
ENABLE_DRAWDOWN_CIRCUIT_BREAKER = True
ENABLE_WAVELET   = True
test_mode        = True            # set False for full universe
ENABLE_PLOTS     = False
LIVE_MODE        = False           # set True to run simple live/paper loop
SIM_LATENCY_MS   = 0               # broker latency simulation; 0 = off
BROKER           = "log"           # "log" = do not place orders, just log
MAX_WORKERS = 1

if LIVE_MODE and FEATURES_FALLBACK_ACTIVE:
    raise RuntimeError(
        "LIVE_MODE=True but compute_enhanced_features fallback is active. "
        "Fix the import so live features match training."
    )


BASE_RESULTS_DIR = "/content/drive/MyDrive/Results_May_2025"
RUN_TAG = datetime.now().strftime("%Y%m%d_%H%M")

RUN_RESULTS_DIR = os.path.join(BASE_RESULTS_DIR, f"ppo_walkforward_results_{RUN_TAG}")
FINAL_MODEL_DIR = os.path.join(BASE_RESULTS_DIR, "ppo_models_master")
QC_TOP_DIR      = os.path.join(BASE_RESULTS_DIR, "ppo_models_QC_TOP")

os.makedirs(QC_TOP_DIR, exist_ok=True)
os.makedirs(RUN_RESULTS_DIR, exist_ok=True)
os.makedirs(FINAL_MODEL_DIR, exist_ok=True)

# Aggregated selector outputs
SELECTOR_FULL_PATH = os.path.join(BASE_RESULTS_DIR, "ppo_model_selector_FULL.csv")
SELECTOR_JSON_PATH = os.path.join(BASE_RESULTS_DIR, "ppo_model_selector_final.json")
MODEL_NAME = "PPO"

# Global skip aggregation (thread-safe)
SKIP_AGG_PATH = os.path.join(RUN_RESULTS_DIR, "skipped_windows_global.csv")
SKIP_LOCK = Lock()

# Logging Setup
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    force=True
)

import hashlib

RUN_TAG = datetime.now().strftime("%Y%m%d_%H%M%S")
MANIFEST_PATH = os.path.join(DRIVE_DIR, f"download_manifest_{RUN_TAG}.json")

ENABLE_HASHES = True  # set False if you want faster runs

def _sha256_file(path: str, chunk_size: int = 1024 * 1024) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        while True:
            b = f.read(chunk_size)
            if not b:
                break
            h.update(b)
    return h.hexdigest()

def _safe_json_dump(obj, path: str):
    tmp = path + ".tmp"
    with open(tmp, "w") as f:
        json.dump(obj, f, indent=2, default=str)
    os.replace(tmp, path)

MANIFEST = {
    "run_tag": RUN_TAG,
    "created_at_utc": datetime.now(timezone.utc).isoformat(),
    "script": "download_feature_engineering",
    "config": {
        "TEST_MODE": TEST_MODE,
        "SYMBOLS": SYMBOLS,
        "INTERVAL": INTERVAL,
        "PERIOD_DAYS": PERIOD_DAYS,
        "FWD_HORIZON": FWD_HORIZON,
        "UP_THR": UP_THR,
        "DN_THR": DN_THR,
        "VAL_FRACTION": VAL_FRACTION,
        "USE_REGIME": USE_REGIME,
        "USE_SENTIMENT": USE_SENTIMENT,
    },
    "environment": {
        "pandas": getattr(pd, "__version__", None),
        "numpy": getattr(np, "__version__", None),
        "yfinance": getattr(yf, "__version__", None),
    },
    "artifacts": {},
}

def manifest_record(name: str, path: str):
    rec = {
        "path": path,
        "exists": os.path.exists(path),
        "size_bytes": os.path.getsize(path) if os.path.exists(path) else None,
        "modified_utc": datetime.fromtimestamp(os.path.getmtime(path), tz=timezone.utc).isoformat()
                        if os.path.exists(path) else None,
    }
    if ENABLE_HASHES and rec["exists"]:
        try:
            rec["sha256"] = _sha256_file(path)
        except Exception as e:
            rec["sha256_error"] = str(e)

    MANIFEST["artifacts"][name] = rec
    _safe_json_dump(MANIFEST, MANIFEST_PATH)
    logging.info(f"[MANIFEST] recorded {name} -> {path}")

# =========================
# 0) Artifact + Repro Hardening
# =========================
import platform, hashlib
from dataclasses import asdict
from datetime import timezone

SEED = 42
set_random_seed(SEED)

# Toggle hashing (sha256). Costs a bit of time; leave False if you want speed.
ENABLE_HASHES = False

def _sha256_file(path: str, chunk_size: int = 1024 * 1024) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        while True:
            b = f.read(chunk_size)
            if not b:
                break
            h.update(b)
    return h.hexdigest()

def _safe_json_dump(obj, path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    tmp = path + ".tmp"
    with open(tmp, "w") as f:
        json.dump(obj, f, indent=2, default=str)
    os.replace(tmp, path)

def now_utc():
    return datetime.now(timezone.utc)

def build_run_config() -> dict:
    cfg = {
        "run_tag": globals().get("RUN_TAG"),
        "run_results_dir": globals().get("RUN_RESULTS_DIR"),
        "final_model_dir": globals().get("FINAL_MODEL_DIR"),
        "qc_top_dir": globals().get("QC_TOP_DIR"),
        "base_results_dir": globals().get("BASE_RESULTS_DIR"),

        "seed": globals().get("SEED"),
        "device": globals().get("DEVICE_STR"),
        "use_gpu": bool(globals().get("USE_GPU", False)),

        "flags": {
            "ENABLE_SENTIMENT": globals().get("ENABLE_SENTIMENT", False),
            "ENABLE_DRAWDOWN_CIRCUIT_BREAKER": globals().get("ENABLE_DRAWDOWN_CIRCUIT_BREAKER", False),
            "ENABLE_WAVELET": globals().get("ENABLE_WAVELET", False),
            "ENABLE_PLOTS": globals().get("ENABLE_PLOTS", False),
            "LIVE_MODE": globals().get("LIVE_MODE", False),
            "SIM_LATENCY_MS": globals().get("SIM_LATENCY_MS", 0),
            "BROKER": globals().get("BROKER", "log"),
            "test_mode": globals().get("test_mode", False),
            "max_workers": globals().get("MAX_WORKERS", 1),
            "features_fallback_active": globals().get("FEATURES_FALLBACK_ACTIVE", False),
        },

        "walkforward": {
            "WINDOW_SIZE": globals().get("WINDOW_SIZE"),
            "STEP_SIZE": globals().get("STEP_SIZE"),
            "TIMESTEPS": globals().get("TIMESTEPS"),
            "TOP_N_WINDOWS": globals().get("TOP_N_WINDOWS"),
        },

        "env_kwargs": dict(globals().get("ENV_KWARGS", {})),

        "ppo_param_profiles": {
            "FAST": dict(globals().get("FAST", {})) if isinstance(globals().get("FAST", {}), dict) else {},
            "SLOW": dict(globals().get("SLOW", {})) if isinstance(globals().get("SLOW", {}), dict) else {},
            "fast_names": sorted(list(globals().get("fast_names", []))),
            "slow_names": sorted(list(globals().get("slow_names", []))),
        },

        "environment": {
            "python": platform.python_version(),
            "platform": platform.platform(),
            "torch": getattr(torch, "__version__", None),
            "gymnasium": getattr(gym, "__version__", None),
            "stable_baselines3": None,
            "yfinance": getattr(yf, "__version__", None),
            "pandas": getattr(pd, "__version__", None),
            "numpy": getattr(np, "__version__", None),
        }
    }

    try:
        import stable_baselines3
        cfg["environment"]["stable_baselines3"] = stable_baselines3.__version__
    except Exception:
        pass

    return cfg

# Artifact manifest (run-level)
ARTIFACT_MANIFEST_PATH = os.path.join(RUN_RESULTS_DIR, "artifact_manifest.json")

def load_manifest() -> dict:
    if os.path.exists(ARTIFACT_MANIFEST_PATH):
        try:
            with open(ARTIFACT_MANIFEST_PATH, "r") as f:
                return json.load(f)
        except Exception:
            pass
    return {
        "run_tag": RUN_TAG,
        "run_results_dir": RUN_RESULTS_DIR,
        "final_model_dir": FINAL_MODEL_DIR,
        "qc_top_dir": QC_TOP_DIR,
        "created_at": now_utc().isoformat(),
        "artifacts": {}  # prefix -> dict
    }

MANIFEST = load_manifest()

def manifest_update(prefix: str, updates: dict):
    """Update manifest in memory + flush to disk (atomic)."""
    MANIFEST["artifacts"].setdefault(prefix, {})
    MANIFEST["artifacts"][prefix].update(updates)
    _safe_json_dump(MANIFEST, ARTIFACT_MANIFEST_PATH)

def manifest_record_paths(prefix: str, paths: dict, do_hashes: bool = False):
    """Record paths, exists flags, and optional sha256 hashes."""
    rec = {}
    for k, p in paths.items():
        rec[k] = {
            "path": p,
            "exists": bool(p and os.path.exists(p)),
        }
        if do_hashes and p and os.path.exists(p):
            try:
                rec[k]["sha256"] = _sha256_file(p)
            except Exception as e:
                rec[k]["sha256_error"] = str(e)
    manifest_update(prefix, {"paths": rec, "updated_at": now_utc().isoformat()})

def write_master_bundle(prefix: str,
                        features: list,
                        threshold: float,
                        model_info: dict):
    """
    Writes the required MASTER artifacts into FINAL_MODEL_DIR for *every window*:
      - {prefix}_features.json
      - {prefix}_probability_config.json
      - {prefix}_model_info.json
    """
    f_path = os.path.join(FINAL_MODEL_DIR, f"{prefix}_features.json")
    p_path = os.path.join(FINAL_MODEL_DIR, f"{prefix}_probability_config.json")
    i_path = os.path.join(FINAL_MODEL_DIR, f"{prefix}_model_info.json")

    _safe_json_dump({"features": list(features)}, f_path)
    _safe_json_dump(
        {"threshold": float(threshold), "use_confidence": True, "inference_mode": "deterministic"},
        p_path
    )
    _safe_json_dump(model_info, i_path)

    manifest_record_paths(
        prefix,
        {
            "master_features_json": f_path,
            "master_probability_config_json": p_path,
            "master_model_info_json": i_path,
        },
        do_hashes=ENABLE_HASHES
    )

def health_check_latest_run(base_dir: str) -> dict:
    """
    One unified health check:
    - finds newest ppo_walkforward_results_* folder
    - reads any summary*.csv
    - reports counts and latest timestamp
    """
    run_dirs = sorted(glob.glob(os.path.join(base_dir, "ppo_walkforward_results_*")))[::-1]
    if not run_dirs:
        return {"ok": False, "reason": "No run folders found."}

    latest = run_dirs[0]
    summaries = sorted(glob.glob(os.path.join(latest, "summary*.csv")))
    out = {
        "ok": True,
        "latest_run_dir": latest,
        "summary_files": summaries,
        "rows_total": 0,
        "tickers": [],
    }

    frames = []
    for p in summaries:
        try:
            tmp = pd.read_csv(p)
            frames.append(tmp)
        except Exception:
            pass

    if frames:
        combo = pd.concat(frames, ignore_index=True)
        out["rows_total"] = int(len(combo))
        if "Ticker" in combo.columns:
            out["tickers"] = sorted(list(combo["Ticker"].dropna().astype(str).unique()))
    else:
        out["ok"] = False
        out["reason"] = "Latest run has no readable summary*.csv"

    return out

# ---- Device / parallel safety gate ----
USE_GPU = torch.cuda.is_available() and (MAX_WORKERS == 1) and (not test_mode)
DEVICE_STR = "cuda" if USE_GPU else "cpu"

# Write run config once, at startup
logging.info(
    f"[CONFIG] MAX_WORKERS={MAX_WORKERS} | cuda_avail={torch.cuda.is_available()} "
    f"| test_mode={test_mode} | USE_GPU={USE_GPU} | DEVICE_STR={DEVICE_STR}"
)

# Write run config once, at startup
RUN_CONFIG_PATH = os.path.join(RUN_RESULTS_DIR, "config.json")
_safe_json_dump(build_run_config(), RUN_CONFIG_PATH)
logging.info(f"[CONFIG] Wrote run config → {RUN_CONFIG_PATH}")

# Warn once if user tries to use LIVE_MODE with fallback features
if LIVE_MODE and FEATURES_FALLBACK_ACTIVE:
    logging.warning(
        "LIVE_MODE=True but compute_enhanced_features fallback is active. "
        "Live inference may not match training features/VecNormalize."
    )

# Global training settings
WINDOW_SIZE = 3500
STEP_SIZE   = 500
TIMESTEPS   = 150_000  # overridden in test_mode block to smaller value


DATA_PATH = "multi_stock_feature_engineered_dataset.csv"
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError("Required feature-engineered dataset not found!")

df = pd.read_csv(DATA_PATH)
df["Datetime"] = pd.to_datetime(df["Datetime"])

RUN_CONFIG_PATH = os.path.join(RUN_RESULTS_DIR, "config.json")
FEATURE_MANIFEST_PATH = os.path.join(RUN_RESULTS_DIR, "feature_manifest.json")
DATA_PROVENANCE_PATH  = os.path.join(RUN_RESULTS_DIR, "data_provenance.json")

def _sha256_file(path: str, chunk_size: int = 1024 * 1024) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        while True:
            b = f.read(chunk_size)
            if not b:
                break
            h.update(b)
    return h.hexdigest()

def build_ordered_feature_list(df_in: pd.DataFrame) -> list[str]:
    # exclude identifiers/targets
    exclude = {"Datetime", "Symbol", "Target", "Return"}
    feats = []
    for c in df_in.columns:
        if c in exclude:
            continue
        if pd.api.types.is_numeric_dtype(df_in[c]):
            feats.append(str(c))
    # stable ordering: alphabetical (or keep a canonical list you control)
    feats = sorted(feats)
    return feats

def build_data_provenance_from_df(df_in: pd.DataFrame) -> dict:
    prov = {
        "source": "prebuilt_csv",
        "data_path": os.path.abspath(DATA_PATH),
        "data_sha256": _sha256_file(DATA_PATH) if os.path.exists(DATA_PATH) else None,
        "rows_total": int(len(df_in)),
        "symbols": sorted(df_in["Symbol"].dropna().astype(str).unique().tolist()) if "Symbol" in df_in.columns else [],
        "interval": globals().get("INTERVAL", None),         # if present in your globals
        "period_days": globals().get("PERIOD_DAYS", None),   # if present in your globals
        "notes": "Derived from feature-engineered dataset used for training",
        "per_symbol": {}
    }

    if "Symbol" in df_in.columns and "Datetime" in df_in.columns:
        for sym, g in df_in.groupby("Symbol", sort=False):
            g2 = g.sort_values("Datetime")
            prov["per_symbol"][str(sym)] = {
                "rows": int(len(g2)),
                "start": str(g2["Datetime"].iloc[0]),
                "end": str(g2["Datetime"].iloc[-1]),
            }
    return prov

def build_run_config() -> dict:
    cfg = {
        "run_tag": RUN_TAG,
        "created_utc": datetime.now(timezone.utc).isoformat(),
        "paths": {
            "run_results_dir": RUN_RESULTS_DIR,
            "final_model_dir": FINAL_MODEL_DIR,
            "qc_top_dir": QC_TOP_DIR,
            "base_results_dir": BASE_RESULTS_DIR,
        },
        "seed": SEED,
        "device": DEVICE_STR,
        "flags": {
            "ENABLE_VOL_CONTROLS": ENABLE_VOL_CONTROLS,
            "ENABLE_SENTIMENT": ENABLE_SENTIMENT,
            "ENABLE_WAVELET": ENABLE_WAVELET,
            "ENABLE_DRAWDOWN_CIRCUIT_BREAKER": ENABLE_DRAWDOWN_CIRCUIT_BREAKER,
            "LIVE_MODE": LIVE_MODE,
            "test_mode": test_mode,
            "MAX_WORKERS": MAX_WORKERS,
        },
        "walkforward": {
            "WINDOW_SIZE": WINDOW_SIZE,
            "STEP_SIZE": STEP_SIZE,
            "TIMESTEPS": TIMESTEPS,
            "TOP_N_WINDOWS": TOP_N_WINDOWS,
        },
        "env_kwargs": dict(ENV_KWARGS),
        "environment": {
            "python": platform.python_version(),
            "platform": platform.platform(),
            "torch": getattr(torch, "__version__", None),
            "gymnasium": getattr(gym, "__version__", None),
            "stable_baselines3": None,
            "pandas": getattr(pd, "__version__", None),
            "numpy": getattr(np, "__version__", None),
            "yfinance": getattr(yf, "__version__", None),
        }
    }
    try:
        import stable_baselines3
        cfg["environment"]["stable_baselines3"] = stable_baselines3.__version__
    except Exception:
        pass
    return cfg

# Write run config once
_safe_json_dump(build_run_config(), RUN_CONFIG_PATH)
logging.info(f"[PHASE0] Wrote run config → {RUN_CONFIG_PATH}")

# Ordered feature manifest (stable feature list)
ORDERED_FEATURES = build_ordered_feature_list(df)
_safe_json_dump({"features_ordered": ORDERED_FEATURES}, FEATURE_MANIFEST_PATH)
logging.info(f"[PHASE0] Wrote ordered feature manifest → {FEATURE_MANIFEST_PATH}")

# Data provenance from the dataset actually used
PROVENANCE = build_data_provenance_from_df(df)
_safe_json_dump(PROVENANCE, DATA_PROVENANCE_PATH)
logging.info(f"[PHASE0] Wrote data provenance → {DATA_PROVENANCE_PATH}")

# Record these into your artifact manifest too (if you want)
manifest_record_paths(
    prefix="RUN_LEVEL",
    paths={
        "config_json": RUN_CONFIG_PATH,
        "feature_manifest_json": FEATURE_MANIFEST_PATH,
        "data_provenance_json": DATA_PROVENANCE_PATH,
        "dataset_csv": os.path.abspath(DATA_PATH),
    },
    do_hashes=ENABLE_HASHES
)

# Wavelet fallback
if ENABLE_WAVELET and "Denoised_Close" not in df.columns:
    logging.warning("ENABLE_WAVELET=True but 'Denoised_Close' missing; "
                    "falling back to Close->Denoised_Close.")
    df["Denoised_Close"] = df["Close"]


def record_skips_global(ticker: str, skipped_windows: list,
                        total_windows: int = None, fully_skipped: bool = False):
    """Append skipped windows to the global skip log."""
    if not skipped_windows and not fully_skipped:
        return
    import csv
    with SKIP_LOCK:
        new_file = not os.path.exists(SKIP_AGG_PATH)
        with open(SKIP_AGG_PATH, "a", newline="") as f:
            w = csv.writer(f)
            if new_file:
                w.writerow(["Ticker", "Window", "FullySkipped", "TotalWindows"])
            if fully_skipped:
                w.writerow([ticker, "ALL", True, total_windows if total_windows is not None else ""])
            else:
                for wname in skipped_windows:
                    try:
                        _, win_str = wname.split("_window")
                        win = int(win_str)
                    except Exception:
                        win = ""
                    w.writerow([ticker, win, False, total_windows if total_windows is not None else ""])

ENV_KWARGS = dict(
    window_size=10,
    cost_rate=0.0002,
    slip_rate=0.0003,

    k_alpha=0.0,
    k_mom=0.15,
    k_sent=(0.01 if ENABLE_SENTIMENT else 0.0),
    mom_source="denoised",
    mom_lookback=20,

    min_trade_delta=0.08,
    cooldown=10,

    reward_clip=0.05,
    k_vol=0.00,
    k_dd=0.00,

    # --- SLO / risk control ---
    enable_slo=ENABLE_DRAWDOWN_CIRCUIT_BREAKER,
    slo_max_dd=0.10,          # 10% max drawdown circuit breaker
    slo_lock_steps=25,        # lock trading N steps after SLO triggers
    slo_hard_stop_enabled=False,
    slo_hard_stop_pct=0.01,   # 1% adverse one-bar move triggers flatten (if enabled)

    slo_chop_enabled=True,
    slo_chop_col="Chop",          # column computed per window
    slo_chop_lock_steps=5,        # lock a few steps once chop triggers

        # --- Volatility controls ---
    enable_vol_controls=ENABLE_VOL_CONTROLS,
    vol_col="Vol_EWM",                 # we will compute this per window
    vol_gate_quantile=VOL_GATE_Q,      # gate threshold within each window
    vol_scale_k=VOL_SCALE_K,
    vol_min_scale=VOL_MIN_SCALE,

    tp_base=TP_BASE,
    sl_base=SL_BASE,
    tp_min=TP_MIN,
    tp_max=TP_MAX,
    sl_min=SL_MIN,
    sl_max=SL_MAX,

)


class ContinuousPositionEnv(StocksEnv):
    """
    Continuous action PPO wrapper around gym-anytrading StocksEnv.

    Key design:
    - We implement our own continuous position + costs + shaped reward + SLO risk control.
    - We STILL call super().step(0) once per step to advance the parent env tick and generate obs/done.
      We IGNORE the parent reward, so PPO learns from our custom reward.
    """

    def __init__(self, df, frame_bound, **kwargs):
        # Require window_size from ENV_KWARGS
        if "window_size" not in kwargs:
            raise ValueError("ContinuousPositionEnv requires window_size (pass via ENV_KWARGS).")

        window_size = int(kwargs.pop("window_size"))

        # Pull params
        cost_rate       = float(kwargs.pop("cost_rate", 0.0002))
        slip_rate       = float(kwargs.pop("slip_rate", 0.0003))
        k_alpha         = float(kwargs.pop("k_alpha", 0.0))
        k_mom           = float(kwargs.pop("k_mom", 0.15))
        k_sent          = float(kwargs.pop("k_sent", 0.0))
        mom_source      = str(kwargs.pop("mom_source", "denoised"))
        mom_lookback    = int(kwargs.pop("mom_lookback", 20))
        min_trade_delta = float(kwargs.pop("min_trade_delta", 0.04))
        cooldown        = int(kwargs.pop("cooldown", 6))
        reward_clip     = float(kwargs.pop("reward_clip", 0.05))
        k_vol           = float(kwargs.pop("k_vol", 0.0))
        k_dd            = float(kwargs.pop("k_dd", 0.0))
        enable_vol_controls = bool(kwargs.pop("enable_vol_controls", False))
        vol_col             = str(kwargs.pop("vol_col", "Vol_EWM"))
        vol_gate_quantile   = float(kwargs.pop("vol_gate_quantile", 0.95))
        vol_scale_k         = float(kwargs.pop("vol_scale_k", 2.0))
        vol_min_scale       = float(kwargs.pop("vol_min_scale", 0.25))

        tp_base = float(kwargs.pop("tp_base", 0.03))
        sl_base = float(kwargs.pop("sl_base", 0.02))
        tp_min  = float(kwargs.pop("tp_min", 0.01))
        tp_max  = float(kwargs.pop("tp_max", 0.08))
        sl_min  = float(kwargs.pop("sl_min", 0.005))
        sl_max  = float(kwargs.pop("sl_max", 0.06))

        # --- SLO / risk control params ---
        enable_slo            = bool(kwargs.pop("enable_slo", False))
        slo_max_dd            = float(kwargs.pop("slo_max_dd", 0.10))
        slo_lock_steps        = int(kwargs.pop("slo_lock_steps", 25))
        slo_hard_stop_enabled = bool(kwargs.pop("slo_hard_stop_enabled", False))
        slo_hard_stop_pct     = float(kwargs.pop("slo_hard_stop_pct", 0.01))

        # --- Chop lock params ---
        slo_chop_enabled    = bool(kwargs.pop("slo_chop_enabled", False))
        slo_chop_col        = str(kwargs.pop("slo_chop_col", "Chop"))
        slo_chop_lock_steps = int(kwargs.pop("slo_chop_lock_steps", 5))

        self.slo_chop_enabled    = slo_chop_enabled
        self.slo_chop_col        = slo_chop_col
        self.slo_chop_lock_steps = max(0, slo_chop_lock_steps)
        self._chop_lock_until    = -1

        # Fail fast on unexpected env kwargs
        if kwargs:
            raise ValueError(f"Unexpected env kwargs: {list(kwargs.keys())}")

        super().__init__(
            df=df.reset_index(drop=True),
            frame_bound=frame_bound,
            window_size=window_size
        )

        self._parent_action_space = self.action_space

        logging.info(f"[ENV] Parent action_space = {self.action_space} | observation_space = {self.observation_space}")

        # Gymnasium compatibility wrapper (sometimes needed in Colab/SB3)
        if isinstance(self.observation_space, gym.spaces.Box):
            self.observation_space = GBox(
                low=self.observation_space.low,
                high=self.observation_space.high,
                shape=self.observation_space.shape,
                dtype=self.observation_space.dtype,
            )

        # --- Store params ---
        self.cost_rate       = cost_rate
        self.slip_rate       = slip_rate
        self.k_alpha         = k_alpha
        self.k_mom           = k_mom
        self.k_sent          = k_sent
        self.mom_source      = mom_source
        self.mom_lookback    = mom_lookback
        self.min_trade_delta = min_trade_delta
        self.cooldown        = cooldown
        self.reward_clip     = reward_clip
        self.k_vol           = k_vol
        self.k_dd            = k_dd

        # --- VOL controls (store on self) ---
        self.enable_vol_controls = enable_vol_controls
        self.vol_col             = vol_col
        self.vol_gate_quantile   = vol_gate_quantile
        self.vol_scale_k         = vol_scale_k
        self.vol_min_scale       = vol_min_scale

        # --- TP/SL params (store on self) ---
        self.tp_base = tp_base
        self.sl_base = sl_base
        self.tp_min  = tp_min
        self.tp_max  = tp_max
        self.sl_min  = sl_min
        self.sl_max  = sl_max


        # --- SLO ---
        self.enable_slo            = enable_slo
        self.slo_max_dd            = slo_max_dd
        self.slo_lock_steps        = max(0, slo_lock_steps)
        self.slo_hard_stop_enabled = slo_hard_stop_enabled
        self.slo_hard_stop_pct     = max(0.0, slo_hard_stop_pct)
        self._slo_lock_until       = -1  # tick index until which trading is locked

        # --- Portfolio state ---
        self.nav            = 1.0
        self.pos            = 0.0  # continuous position in [-1, 1]
        self.peak_nav       = 1.0
        self.nav_history    = []
        self.ret_history    = []
        self.trade_count    = 0
        self._last_trade_step = -self.cooldown

        # Continuous action for PPO
        self.action_space = GBox(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)

    def reset(self, **kwargs):
        out = super().reset(**kwargs)
        if isinstance(out, tuple):
            obs, info = out
        else:
            obs, info = out, {}

        # Reset custom state
        self.nav = 1.0
        self.pos = 0.0
        self.peak_nav = 1.0
        self.nav_history = [self.nav]
        self.ret_history = []
        self.trade_count = 0
        self._last_trade_step = -self.cooldown
        self._slo_lock_until = -1

        # Entry tracking for TP/SL
        self.entry_price = None
        self.entry_tick  = None
        self._chop_lock_until = -1

        # Precompute window-level reference + gate from df if available
        try:
            self._vol_ref = float(self.df["Vol_Ref"].iloc[0]) if "Vol_Ref" in self.df.columns else 0.0
        except Exception:
            self._vol_ref = 0.0

        try:
            self._vol_gate = float(self.df["Vol_Gate"].iloc[0]) if "Vol_Gate" in self.df.columns else 0.0
        except Exception:
            self._vol_gate = 0.0

        info = info or {}
        info.update({
            "nav": self.nav,
            "pos": self.pos,
            "trade_count": int(self.trade_count),
            "slo_locked": False,
            "slo_lock_until": int(self._slo_lock_until),
            "target_pos": 0.0,
        })
        return obs, info

    def _ret_t(self):
        """
        One-bar return at the CURRENT tick (based on df + _current_tick).
        This is aligned with the parent env tick, since we advance via super().step(0).
        """
        cur = float(self.df.loc[self._current_tick, "Close"])
        prev = float(self.df.loc[max(self._current_tick - 1, 0), "Close"])
        return 0.0 if prev <= 0 else (cur - prev) / prev

    def _mom_signal(self):
        # Optional MACD momentum
        if self.mom_source == "macd" and "MACD_Line" in self.df.columns:
            recent = self.df["MACD_Line"].iloc[max(self._current_tick - 200, 0):self._current_tick + 1]
            denom = 1e-6 + float(recent.std())
            return float(np.tanh(float(self.df.loc[self._current_tick, "MACD_Line"]) / denom))

        # Default: denoised slope momentum
        if "Denoised_Close" in self.df.columns and self._current_tick - self.mom_lookback >= 0:
            now  = float(self.df.loc[self._current_tick, "Denoised_Close"])
            then = float(self.df.loc[self._current_tick - self.mom_lookback, "Denoised_Close"])
            base = float(self.df.loc[max(self._current_tick - 1, 0), "Close"])
            slope = (now - then) / max(self.mom_lookback, 1)
            return float(np.tanh(10.0 * (slope / max(abs(base), 1e-6))))

        return 0.0
    def _price_t(self) -> float:
        return float(self.df.loc[self._current_tick, "Close"])

    def _vol_t(self) -> float:
        if not getattr(self, "enable_vol_controls", False):
            return 0.0
        col = getattr(self, "vol_col", "Vol_EWM")
        if col in self.df.columns:
            try:
                return float(self.df.loc[self._current_tick, col])
            except Exception:
                return 0.0
        return 0.0

    def _dynamic_tp_sl(self, vol_t: float) -> tuple[float, float]:
        """
        Wider in low vol, tighter in high vol.
        We scale TP/SL by (vol_ref / vol_t), clipped to bounds.
        """
        vol_ref = float(getattr(self, "_vol_ref", 0.0))
        eps = 1e-9
        ratio = (vol_ref / max(vol_t, eps)) if vol_ref > 0 else 1.0

        tp = float(np.clip(self.tp_base * ratio, self.tp_min, self.tp_max))
        sl = float(np.clip(self.sl_base * ratio, self.sl_min, self.sl_max))
        return tp, sl

    def _base_ret_next(self) -> float:
        """
        Baseline return for the bar that just completed AFTER advancing parent env.
        Uses current tick vs previous tick close.
        """
        try:
            cur = float(self.df.loc[self._current_tick, "Close"])
            prev = float(self.df.loc[max(self._current_tick - 1, 0), "Close"])
            return 0.0 if prev <= 0 else (cur - prev) / prev
        except Exception:
            return 0.0

    def _step_parent_hold(self):
        # temporarily swap action_space back so parent validation passes
        cur_as = self.action_space
        try:
            self.action_space = self._parent_action_space
            out = super().step(0)  # HOLD for the parent env
        finally:
            self.action_space = cur_as

        # gym-anytrading may return 4-tuple (gym) or 5-tuple (gymnasium)
        if len(out) == 5:
            obs, _, terminated, truncated, info = out
            return obs, terminated, truncated, info
        elif len(out) == 4:
            obs, _, done, info = out
            return obs, bool(done), False, info
        else:
            raise ValueError(f"Unexpected parent step() return length: {len(out)}")

    def step(self, action):
        a = float(np.array(action).squeeze())
        next_pos = float(np.clip(a, -1.0, 1.0))

        # defaults so they're ALWAYS defined
        forced_close = False
        tp_pct = 0.0
        sl_pct = 0.0
        scale = 1.0
        gated = False
        slo_triggered = False

        # bar return + vol (must be defined early)
        r_t = self._ret_t()
        vol_t = self._vol_t()

        # --- TP/SL forced flatten (computed BEFORE SLO lock check) ---
        if self.enable_vol_controls and (self.pos != 0.0) and (self.entry_price is not None):
            tp_pct, sl_pct = self._dynamic_tp_sl(vol_t)
            px = self._price_t()
            pnl_dir = ((px - float(self.entry_price)) / max(float(self.entry_price), 1e-9)) * float(np.sign(self.pos))

            if pnl_dir >= tp_pct:
                next_pos = 0.0
                forced_close = True
            elif pnl_dir <= -sl_pct:
                next_pos = 0.0
                forced_close = True

        # --- SLO lock (don’t block forced TP/SL flatten) ---
        locked = (
            getattr(self, "enable_slo", False)
            and getattr(self, "_current_tick", 0) < getattr(self, "_slo_lock_until", -1)
        )
        if locked and not forced_close:
            next_pos = float(self.pos)

        # =========================
        # Volatility controls: scale + gate
        # =========================
        vol_ref  = float(getattr(self, "_vol_ref", 0.0))
        vol_gate = float(getattr(self, "_vol_gate", 0.0))

        # scale down exposure when vol > ref
        if self.enable_vol_controls and vol_ref > 0 and vol_t > vol_ref:
            scale = 1.0 / (1.0 + self.vol_scale_k * max(0.0, (vol_t / (vol_ref + 1e-9)) - 1.0))
            scale = float(np.clip(scale, self.vol_min_scale, 1.0))
            next_pos = float(np.clip(next_pos * scale, -1.0, 1.0))

        # gate new risk when vol extreme (do not block TP/SL forced_close)
        if self.enable_vol_controls and vol_gate > 0 and vol_t >= vol_gate and not forced_close:
            gated = True
            if abs(self.pos) < 1e-9:
                next_pos = 0.0
            else:
                if np.sign(next_pos) != np.sign(self.pos):
                    next_pos = 0.0
                elif abs(next_pos) > abs(self.pos):
                    next_pos = float(self.pos)

        chop_now = False
        if getattr(self, "slo_chop_enabled", False) and getattr(self, "slo_chop_col", "") in self.df.columns:
            try:
                chop_now = bool(int(self.df.loc[self._current_tick, self.slo_chop_col]) == 1)
            except Exception:
                chop_now = False

        # if chop triggers, lock for N steps and force flat
        if chop_now:
            self._chop_lock_until = max(self._chop_lock_until, self._current_tick + int(self.slo_chop_lock_steps))

        chop_locked = (self._current_tick < self._chop_lock_until)

        if chop_locked and not forced_close:
            # ignore PPO action: must be flat (or only allow reducing risk)
            if abs(self.pos) < 1e-9:
                next_pos = 0.0
            else:
                # allow reduction only (move toward 0), block adds/reversals
                if np.sign(next_pos) != np.sign(self.pos):
                    next_pos = 0.0
                elif abs(next_pos) > abs(self.pos):
                    next_pos = float(self.pos)


        # --- friction / cooldown gating (single source of truth) ---
        if forced_close:
            changed = (abs(self.pos) > 1e-9)  # always allow flatten on TP/SL
        else:
            changed = (
                (abs(next_pos - self.pos) >= self.min_trade_delta)
                and ((self._current_tick - self._last_trade_step) >= self.cooldown)
            )

        # if vol-gated, cancel changes (and cancel costs) unless forced_close
        if gated and not forced_close:
            changed = False

        delta_pos = (next_pos - self.pos) if changed else 0.0
        fee_cost  = self.cost_rate * abs(delta_pos)
        slip_cost = self.slip_rate * abs(delta_pos)
        trade_cost = fee_cost + slip_cost

        # turnover proxy (continuous): abs change in position
        turnover_step = abs(delta_pos)

        # --- one-bar PnL from previous position ---
        pnl_from_prev_pos = self.pos * r_t

        # --- shaped reward uses FINAL trade_cost (post gate/cooldown) ---
        rel_alpha = pnl_from_prev_pos - r_t
        mom_term  = self.pos * self._mom_signal()
        shaped = pnl_from_prev_pos + (self.k_mom * mom_term) - trade_cost
        reward = float(np.clip(shaped, -self.reward_clip, self.reward_clip))

        # compute nav with FINAL trade_cost (pre-SLO)
        next_nav = self.nav * (1.0 + pnl_from_prev_pos - trade_cost)

        prev_peak = self.peak_nav
        dd_now = 0.0
        if getattr(self, "enable_slo", False):
            dd_now = (prev_peak - next_nav) / max(prev_peak, 1e-9)

        # --- SLO drawdown circuit breaker (BEFORE committing nav) ---
        if getattr(self, "enable_slo", False) and (dd_now >= float(getattr(self, "slo_max_dd", 1e9))):
            slo_triggered = True

            # flatten cost MUST be included BEFORE we commit nav
            if self.pos != 0.0:
                trade_cost += (self.cost_rate * abs(self.pos)) + (self.slip_rate * abs(self.pos))
                next_nav = self.nav * (1.0 + pnl_from_prev_pos - trade_cost)

            # flatten + lock
            self.pos = 0.0
            next_pos = 0.0
            self._slo_lock_until = self._current_tick + int(getattr(self, "slo_lock_steps", 0))
            self._last_trade_step = self._current_tick

            dd_now = (self.peak_nav - next_nav) / max(self.peak_nav, 1e-9)

            changed = False
            delta_pos = 0.0

        # --- commit NAV + peak AFTER SLO logic ---
        self.nav = float(next_nav)
        self.nav_history.append(self.nav)
        self.peak_nav = max(self.peak_nav, self.nav)

        # --- execute trade AFTER NAV update ---
        executed_trade = False
        if changed:
            prev_pos = float(self.pos)
            self.pos = float(next_pos)
            self._last_trade_step = self._current_tick
            self.trade_count += 1
            executed_trade = True

            # Entry tracking
            if abs(self.pos) < 1e-9:
                self.entry_price = None
                self.entry_tick = None
            else:
                if (abs(prev_pos) < 1e-9) or (np.sign(prev_pos) != np.sign(self.pos)):
                    self.entry_price = self._price_t()
                    self.entry_tick = int(self._current_tick)

        # --- advance parent env one tick & get obs/done ---
        obs, terminated, truncated, info = self._step_parent_hold()

        # base_ret aligned to the bar we just advanced into
        base_ret = float(self._base_ret_next())

        info = info or {}
        info.update({
            "ret_t": float(r_t),
            "base_ret": float(base_ret),

            "fee_cost": float(fee_cost),
            "slip_cost": float(slip_cost),
            "turnover_step": float(turnover_step),

            "nav": float(self.nav),
            "pos": float(self.pos),
            "trade_cost": float(trade_cost),
            "pnl_from_prev_pos": float(pnl_from_prev_pos),
            "rel_alpha": float(rel_alpha),
            "mom": float(mom_term),
            "changed": bool(changed),
            "executed_trade": bool(executed_trade),
            "trade_count": int(self.trade_count),
            "delta_pos": float(delta_pos),
            "next_pos": float(next_pos),

            "drawdown": float(dd_now),
            "slo_triggered": bool(slo_triggered),
            "slo_locked": bool(
                getattr(self, "enable_slo", False)
                and getattr(self, "_current_tick", 0) < getattr(self, "_slo_lock_until", -1)
            ),
            "slo_lock_until": int(getattr(self, "_slo_lock_until", -1)),

            "vol_t": float(vol_t),
            "vol_ref": float(vol_ref),
            "vol_gate": float(vol_gate),
            "vol_scale": float(scale),
            "vol_gated": bool(gated),
            "tp_pct": float(tp_pct),
            "sl_pct": float(sl_pct),
            "forced_close": bool(forced_close),

            "chop_now": bool(chop_now),
            "chop_locked": bool(chop_locked),
            "chop_lock_until": int(self._chop_lock_until),
        })
        return obs, reward, terminated, truncated, info

def get_mu_sigma(model, obs):
    """SB3 v2-safe way to get Gaussian policy mean/std for continuous actions."""
    with torch.no_grad():
        obs_t, _ = model.policy.obs_to_tensor(obs)
        features = model.policy.extract_features(obs_t)
        latent_pi, _ = model.policy.mlp_extractor(features)

        mean_actions = model.policy.action_net(latent_pi)
        mu = float(mean_actions.detach().cpu().numpy().reshape(-1).mean())

        # log_std can be vector -> convert safely to one representative scalar
        sigma_vec = model.policy.log_std.exp().detach().cpu().numpy().reshape(-1)
        sigma = float(sigma_vec.mean())  # or sigma_vec[0] if you prefer

    return mu, sigma

def get_walk_forward_windows(df_in, window_size=3500, step_size=500, min_len=1200):
    return [
        (start, start + window_size)
        for start in range(0, len(df_in) - min_len, step_size)
        if start + window_size <= len(df_in)
    ]

def save_quantconnect_model(artifact, prefix, save_dir):
    """Save/copy QC-compatible artifacts into save_dir."""
    import shutil

    os.makedirs(save_dir, exist_ok=True)

    # --- Model zip: save or copy ---
    model_dst = os.path.join(save_dir, f"{prefix}_model.zip")

    model_obj = artifact.get("model", None)
    model_src = artifact.get("model_path", None)

    try:
        if model_obj is not None:
            # Save from in-memory SB3 model
            if not os.path.exists(model_dst):
                model_obj.save(model_dst)

        else:
            # Copy from an existing trained window model zip
            if model_src and os.path.exists(model_src):
                if os.path.abspath(model_src) != os.path.abspath(model_dst):
                    shutil.copyfile(model_src, model_dst)
            else:
                # If neither provided, warn loudly
                if not os.path.exists(model_dst):
                    logging.warning(f"[QC SAVE] Missing model for {prefix}: no model_obj and no valid model_path.")
    except Exception as e:
        logging.warning(f"[QC SAVE] Model handling issue for {prefix}: {e}")

    # --- VecNormalize: copy ---
    vecnorm_src = artifact.get("vecnorm_path")
    if vecnorm_src and os.path.exists(vecnorm_src):
        try:
            vecnorm_dst = os.path.join(save_dir, f"{prefix}_vecnorm.pkl")
            if os.path.abspath(vecnorm_src) != os.path.abspath(vecnorm_dst):
                shutil.copyfile(vecnorm_src, vecnorm_dst)
        except Exception as e:
            logging.warning(f"[QC SAVE] VecNormalize handling issue for {prefix}: {e}")
    else:
        logging.warning(f"[QC SAVE] VecNormalize missing for {prefix}: vecnorm_path not found.")

    # --- Features ---
    try:
        with open(os.path.join(save_dir, f"{prefix}_features.json"), "w") as f:
            json.dump({"features": artifact.get("features", [])}, f)
    except Exception as e:
        logging.warning(f"[QC SAVE] Could not write features.json for {prefix}: {e}")

    # --- Probability config ---
    try:
        thr_window = 0.2
        try:
            thr_window = float(artifact.get("result", {}).get("Action_Threshold", 0.2))
        except Exception:
            thr_window = 0.2

        with open(os.path.join(save_dir, f"{prefix}_probability_config.json"), "w") as f:
            json.dump(
                {"threshold": float(thr_window), "use_confidence": True, "inference_mode": "deterministic"},
                f
            )
    except Exception as e:
        logging.warning(f"[QC SAVE] Could not write probability_config.json for {prefix}: {e}")


    # --- Model info ---
    try:
        r = artifact.get("result", {})
        with open(os.path.join(save_dir, f"{prefix}_model_info.json"), "w") as f:
            json.dump({
                "model": "PPO",
                "ticker": r.get("Ticker"),
                "window": r.get("Window"),
                "date_trained": datetime.today().strftime("%Y-%m-%d"),
                "framework": "stable-baselines3",
                "input_features": artifact.get("features", []),
                "final_portfolio": r.get("PPO_Portfolio"),
                "buy_hold": r.get("BuyHold"),
                "sharpe": r.get("Sharpe"),
            }, f)
    except Exception as e:
        logging.warning(f"[QC SAVE] Could not write model_info.json for {prefix}: {e}")

    logging.info(f"[QC SAVE] Saved QC artifacts for {prefix}")

def load_model_and_env(prefix):
    """Load a trained PPO and create a factory to build a matching env window."""
    model_path = os.path.join(FINAL_MODEL_DIR, f"{prefix}_model.zip")
    vec_path   = os.path.join(FINAL_MODEL_DIR, f"{prefix}_vecnorm.pkl")
    model = PPO.load(model_path, device="cpu")

    def make_env(df_window):
        frame_bound = (50, len(df_window) - 3)
        e = DummyVecEnv([lambda: ContinuousPositionEnv(
            df=df_window, frame_bound=frame_bound, **ENV_KWARGS
        )])
        if os.path.exists(vec_path):
            e = VecNormalize.load(vec_path, e)
        e.training = False
        e.norm_reward = False
        return e

    return model, make_env

def latest_df_for_symbol(symbol, horizon_days=5, interval="1m"):
    """Fetch fresh bars and rebuild features exactly like training."""
    end = now_utc()
    start = end - timedelta(days=horizon_days)

    # yfinance wants date-like strings; keep it simple
    df_live = yf.download(
        symbol,
        start=start.date().isoformat(),
        end=end.date().isoformat(),
        interval=interval,
        progress=False,
        auto_adjust=False,
    )
    if df_live is None or df_live.empty:
        return None
    df_live = df_live.reset_index()
    if "Date" in df_live.columns and "Datetime" not in df_live.columns:
        df_live = df_live.rename(columns={"Date": "Datetime"})
    df_live["Symbol"] = symbol
    df_live = compute_enhanced_features(df_live)
    if ENABLE_WAVELET and "Denoised_Close" not in df_live.columns:
        df_live["Denoised_Close"] = df_live["Close"]
    return df_live

def _download_smoke_test(symbol="AAPL"):
    try:
        d = latest_df_for_symbol(symbol, horizon_days=5, interval="1h")
        if d is None or d.empty:
            logging.warning(f"[DL TEST] {symbol}: empty download.")
            return False
        # sanity: must have Datetime + Close
        ok = ("Datetime" in d.columns) and ("Close" in d.columns)
        logging.info(f"[DL TEST] {symbol}: rows={len(d)} cols={len(d.columns)} ok={ok}")
        return bool(ok)
    except Exception as e:
        logging.warning(f"[DL TEST] {symbol}: failed: {e}")
        return False

def predict_latest(symbol, prefix):
    """Build last window, fast-forward env, call model.predict(), return a signal."""
    # --- load per-model threshold ---
    cfg_path = os.path.join(FINAL_MODEL_DIR, f"{prefix}_probability_config.json")
    thr_window = 0.2
    if os.path.exists(cfg_path):
        try:
            with open(cfg_path, "r") as f:
                thr_window = float(json.load(f).get("threshold", 0.2))
        except Exception:
            thr_window = 0.2

    model, make_env = load_model_and_env(prefix)

    live_df = latest_df_for_symbol(symbol)
    if live_df is None or len(live_df) < 100:
        logging.warning("No fresh data yet for live inference.")
        return None

    df_window = live_df.iloc[-2500:].reset_index(drop=True) if len(live_df) > 2500 else live_df.copy()

    env = make_env(df_window)
    obs = env.reset()
    if isinstance(obs, tuple):
        obs, _ = obs

    # fast-forward with HOLD (VecEnv-safe action shape)
    zero_action = np.zeros((1, 1), dtype=np.float32)
    for _ in range(len(df_window) - 1):
        obs, _, dones, infos = env.step(zero_action)
        if isinstance(dones, (np.ndarray, list, tuple)) and len(dones) and dones[0]:
            break

    # always predict
    action, _ = model.predict(obs, deterministic=True)
    mu, sigma = get_mu_sigma(model, obs)
    a = float(np.array(action).squeeze())

    # --- thresholded signal using loaded thr_window ---
    if a > thr_window:
        signal = "BUY"
    elif a < -thr_window:
        signal = "SELL"
    else:
        signal = "HOLD"

    conf = abs(a)
    ts = df_window["Datetime"].iloc[-1] if "Datetime" in df_window.columns else None
    price = float(df_window["Close"].iloc[-1])

    return dict(
        signal=signal,
        confidence=conf,
        action=a,
        threshold=thr_window,
        ts=ts,
        price=price,
        mu=mu,
        sigma=sigma,
    )

def place_order(signal, qty=1):
    """Stub broker router with latency simulation; logs in Colab."""
    if SIM_LATENCY_MS > 0:
        time.sleep(SIM_LATENCY_MS / 1000.0)
    if BROKER == "log":
        logging.info(f"[PAPER] {signal} x{qty}")
    else:
        logging.info(f"[BROKER={BROKER}] {signal} x{qty} (not implemented)")

def live_loop(symbol, best_prefix):
    """Simple polling loop—set LIVE_MODE=True to run."""
    while LIVE_MODE:
        try:
            pred = predict_latest(symbol, best_prefix)
            if pred:
                logging.info(
                    f"{symbol} {pred['ts']} | {pred['signal']} "
                    f"@ {pred['price']:.2f} (conf {pred['confidence']:.2f})"
                )
                place_order(pred["signal"], qty=1)
        except Exception as e:
            logging.error(f"Live loop error: {e}")
        time.sleep(60)  # Poll each minute

TOP_N_WINDOWS = 3

FAST = {
    "lr": 8e-5,
    "n_steps": 3072,
    "batch": 512,
    "clip": 0.2,
    "ent": 0.01,
}

SLOW = {
    "lr": 3e-5,
    "n_steps": 3072,
    "batch": 512,
    "clip": 0.16,
    "ent": 0.005,
}

fast_names = {
    "TSLA","NVDA","AMD","AVGO","AAPL","MSFT","AMZN","GOOGL","META","ADBE","CRM",
    "INTC","QCOM","TXN","ORCL","NEE","GE","XOM","CVX","LLY","NKE","SBUX"
}
slow_names = {
    "BRK-B","JPM","BAC","JNJ","UNH","MRK","PFE","ABBV","ABT","AMGN","PG","PEP","KO",
    "V","MA","WMT","MCD","TMO","DHR","ACN","IBM","LIN","PM","RTX","UPS","UNP","COST","HD","LOW"
}

def pick_params(symbol: str):
    return FAST if symbol in fast_names else SLOW

def export_qc_top_from_existing(ticker: str, top_n: int = 3):
    """Populate QC_TOP_DIR from summary CSVs when models are skipped. """
    run_dirs = sorted(glob.glob(os.path.join(BASE_RESULTS_DIR, "ppo_walkforward_results_*")))
    run_dirs = run_dirs[::-1]  # newest -> oldest

    summary_files = []
    for rd in run_dirs:
        cand = glob.glob(os.path.join(rd, "summary*.csv"))
        if not cand:
            continue
        # quick check: does ANY file contain ticker?
        ok = False
        for p in cand:
            try:
                tmp = pd.read_csv(p, usecols=["Ticker"])
                if (tmp["Ticker"] == ticker).any():
                    ok = True
                    break
            except Exception:
                pass
        if ok:
            summary_files = cand
            break

    if not summary_files:
        logging.warning(f"[QC_TOP] No summaries containing {ticker} found in any run folder.")
        return

    frames = []
    for p in summary_files:
        try:
            tmp = pd.read_csv(p)
            frames.append(tmp)
        except Exception as e:
            logging.warning(f"[QC_TOP] Could not read {p}: {e}")

    if not frames:
        logging.warning(f"[QC_TOP] Could not read any summary files; cannot export QC_TOP for {ticker}.")
        return

    combo = pd.concat(frames, ignore_index=True)

    if "Ticker" not in combo.columns:
        logging.warning("[QC_TOP] Summary files missing 'Ticker' column; cannot export.")
        return

    combo = combo[combo["Ticker"] == ticker].copy()
    if "Prefix" in combo.columns:
        combo["Prefix"] = combo["Prefix"].astype(str)
        combo = combo.drop_duplicates(subset=["Prefix"], keep="last")

    if combo.empty or "Sharpe" not in combo.columns:
        logging.warning(f"[QC_TOP] No rows for {ticker} in summaries (or missing Sharpe); cannot export QC_TOP.")
        return

    # Ensure Sharpe is numeric so sorting works reliably
    combo["Sharpe"] = pd.to_numeric(combo["Sharpe"], errors="coerce")
    combo = combo.dropna(subset=["Sharpe"])
    if combo.empty:
        logging.warning(f"[QC_TOP] All Sharpe values were non-numeric for {ticker}; cannot export.")
        return

    use_prefix = ("Prefix" in combo.columns) and combo["Prefix"].notna().any()

    if use_prefix:
        # Robust path: use saved Prefix directly
        top = combo.sort_values("Sharpe", ascending=False).head(top_n).copy()
        top["__prefix__"] = top["Prefix"].astype(str)
    else:
        # Fallback: reconstruct WindowIdx (less robust)
        def _window_start(w):
            try:
                s = str(w)
                return int(s.split("-")[0]) if "-" in s else np.nan
            except Exception:
                return np.nan

        combo["WindowStart"] = combo["Window"].apply(_window_start)
        combo = combo.sort_values(["WindowStart"]).reset_index(drop=True)
        combo["WindowIdx"] = combo.groupby("Ticker").cumcount() + 1

        top = combo.sort_values("Sharpe", ascending=False).head(top_n).copy()
        top["__prefix__"] = top["WindowIdx"].apply(lambda widx: f"ppo_{ticker}_window{int(widx)}")

    exported = 0

    for _, r in top.iterrows():
        prefix = str(r["__prefix__"])

        model_path = os.path.join(FINAL_MODEL_DIR, f"{prefix}_model.zip")
        vec_path   = os.path.join(FINAL_MODEL_DIR, f"{prefix}_vecnorm.pkl")

        if not (os.path.exists(model_path) and os.path.exists(vec_path)):
            logging.warning(f"[QC_TOP] Missing model/vecnorm for {prefix}; cannot export.")
            continue

        artifact_for_save = {
            "model": None,
            "model_path": model_path,
            "vecnorm_path": vec_path,
            "features": [],         # ok if unknown; QC can load features elsewhere
            "result": r.to_dict(),  # includes Sharpe, Action_Threshold, etc if present
            "prefix": prefix,
        }
        save_quantconnect_model(artifact_for_save, prefix, QC_TOP_DIR)
        exported += 1

    logging.info(f"[QC_TOP] Exported {exported}/{len(top)} QC artifacts for {ticker}.")

def walkforward_ppo(df_sym, ticker,
                    window_size=3500, step_size=500,
                    timesteps=150_000, learning_rate=1e-4,
                    ppo_overrides=None):

    if ppo_overrides is None:
        ppo_overrides = {}

    if len(df_sym) < window_size:
        logging.warning(
            f"Skipping {ticker}: only {len(df_sym)} rows (min required: {window_size})"
        )
        return []

    results = []
    windows = get_walk_forward_windows(
        df_sym,
        window_size=window_size,
        step_size=step_size
    )
    #TEST MODE: force exactly 1 window per ticker
    if test_mode:
        windows = windows[:1]
    top_heap = []
    skipped_windows = []

    # quick check: all windows already have model+vecnorm?
    all_done = True
    for idx in range(len(windows)):
        prefix = f"ppo_{ticker}_window{idx+1}"
        model_ok   = os.path.exists(os.path.join(FINAL_MODEL_DIR, f"{prefix}_model.zip"))
        vecnorm_ok = os.path.exists(os.path.join(FINAL_MODEL_DIR, f"{prefix}_vecnorm.pkl"))
        feat_ok = os.path.exists(os.path.join(FINAL_MODEL_DIR, f"{prefix}_features.json"))
        prob_ok = os.path.exists(os.path.join(FINAL_MODEL_DIR, f"{prefix}_probability_config.json"))
        info_ok = os.path.exists(os.path.join(FINAL_MODEL_DIR, f"{prefix}_model_info.json"))

        if not (model_ok and vecnorm_ok and feat_ok and prob_ok and info_ok):
            all_done = False
            break

    if all_done:
        logging.info(f"Ticker {ticker} fully skipped (all {len(windows)} windows already complete).")
        record_skips_global(ticker, skipped_windows=[], total_windows=len(windows), fully_skipped=True)

        export_qc_top_from_existing(ticker, top_n=TOP_N_WINDOWS)
        return []



    for w_idx, (start, end) in enumerate(windows):
        window_start_time = time.time()
        gc.collect()

        prefix = f"ppo_{ticker}_window{w_idx+1}"
        model_path   = os.path.join(FINAL_MODEL_DIR, f"{prefix}_model.zip")
        vecnorm_path = os.path.join(FINAL_MODEL_DIR, f"{prefix}_vecnorm.pkl")
        feat_ok = os.path.exists(os.path.join(FINAL_MODEL_DIR, f"{prefix}_features.json"))
        prob_ok = os.path.exists(os.path.join(FINAL_MODEL_DIR, f"{prefix}_probability_config.json"))
        info_ok = os.path.exists(os.path.join(FINAL_MODEL_DIR, f"{prefix}_model_info.json"))

        core_ok = os.path.exists(model_path) and os.path.exists(vecnorm_path)
        master_ok = feat_ok and prob_ok and info_ok

        if core_ok and master_ok:
            logging.info(f"Skipping {ticker} | Window {w_idx+1}, already trained + MASTER complete.")
            skipped_windows.append(f"{ticker}_window{w_idx+1}")
            manifest_update(prefix, {"status": "skipped_complete", "updated_at": now_utc().isoformat()})
            continue

        if core_ok and not master_ok:
            logging.warning(f"[PHASE0] {prefix} core exists but MASTER bundle incomplete → will rebuild MASTER artifacts after eval.")

        missing = []
        if not os.path.exists(model_path):   missing.append("model.zip")
        if not os.path.exists(vecnorm_path): missing.append("vecnorm.pkl")
        logging.info(
            f"Will train {ticker} | Window {w_idx+1} because missing: {', '.join(missing)}"
        )

        df_window = df_sym.iloc[start:end].reset_index(drop=True)
        df_window["Ret_1"] = df_window["Close"].pct_change().fillna(0.0)

        # Rolling vol (std of returns)
        df_window["Vol_Roll"] = df_window["Ret_1"].rolling(VOL_LOOKBACK).std().fillna(0.0)

        # EWM vol (std of returns)
        df_window["Vol_EWM"] = df_window["Ret_1"].ewm(span=VOL_EWM_SPAN, adjust=False).std().fillna(0.0)

        # Reference (median vol) + gate threshold
        vol_ref  = float(df_window["Vol_EWM"].median()) if len(df_window) else 0.0
        vol_gate = float(df_window["Vol_EWM"].quantile(VOL_GATE_Q)) if len(df_window) else 0.0
        df_window["Vol_Ref"]  = vol_ref
        df_window["Vol_Gate"] = vol_gate

        # =========================
        # PHASE 1: Chop / bad-regime detection (force flat)
        # =========================
        df_window["Ret20"] = df_window["Close"].pct_change(20).fillna(0.0)
        df_window["Ret1"]  = df_window["Close"].pct_change(1).fillna(0.0)

        sign_flip = (np.sign(df_window["Ret1"]) != np.sign(df_window["Ret1"].shift(1))).astype(float)
        df_window["FlipRate20"] = sign_flip.rolling(20).mean().fillna(0.0)

        trend_low = (
            df_window["Ret20"].abs()
            < df_window["Ret20"].abs().rolling(200).median().fillna(df_window["Ret20"].abs().median())
        )
        flip_high = (
            df_window["FlipRate20"]
            > df_window["FlipRate20"].rolling(200).median().fillna(df_window["FlipRate20"].median())
        )

        df_window["Chop"] = (trend_low & flip_high).astype(int)

        # keep your existing even-length safety
        if len(df_window) <= 52 or len(df_window) % 2 != 0:
            df_window = df_window.iloc[:-1]

        frame_bound = (50, len(df_window) - 3)

        env = DummyVecEnv([lambda: ContinuousPositionEnv(
            df=df_window, frame_bound=frame_bound, **ENV_KWARGS
        )])

        env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0)

        try:
            model = PPO(
                "MlpPolicy",
                env,
                verbose=0,
                device=DEVICE_STR,
                learning_rate=ppo_overrides.get("lr", learning_rate),
                n_steps=ppo_overrides.get("n_steps", 256),
                batch_size=ppo_overrides.get("batch", 64),
                n_epochs=5,
                gamma=0.99,
                gae_lambda=0.95,
                clip_range=ppo_overrides.get("clip", 0.2),
                ent_coef=ppo_overrides.get("ent", 0.005),
                policy_kwargs=dict(net_arch=[64, 64]),
            )

            logging.info(f"Training {ticker} Window {w_idx+1}/{len(windows)}")
            model.learn(total_timesteps=timesteps)

            # Evaluation pass
            def _unwrap_reset(reset_out):
                """VecEnv reset() returns obs (np.ndarray). Gymnasium env returns (obs, info)."""
                if isinstance(reset_out, tuple) and len(reset_out) == 2:
                    obs, info = reset_out
                    return obs, info
                return reset_out, {}

            def _unwrap_step(step_out):
                """
                DummyVecEnv step() returns: (obs, rewards, dones, infos)
                  obs: np.ndarray shape (n_env, obs_dim)
                  rewards: np.ndarray shape (n_env,)
                  dones: np.ndarray shape (n_env,)
                  infos: list[dict] length n_env
                """
                obs, rewards, dones, infos = step_out

                # rewards
                if isinstance(rewards, np.ndarray):
                    rew0 = float(rewards[0])
                elif isinstance(rewards, (list, tuple)):
                    rew0 = float(rewards[0])
                else:
                    rew0 = float(rewards)

                # dones
                if isinstance(dones, np.ndarray):
                    done0 = bool(dones[0])
                elif isinstance(dones, (list, tuple)):
                    done0 = bool(dones[0])
                else:
                    done0 = bool(dones)

                # infos
                if isinstance(infos, (list, tuple)):
                    info0 = infos[0] if len(infos) else {}
                elif isinstance(infos, dict):
                    info0 = infos
                else:
                    info0 = {}

                return obs, rew0, done0, info0


            # --- Evaluation pass ---
            env.training = False
            env.norm_reward = False

            obs, reset_info = _unwrap_reset(env.reset())

            nav_track = [1.0]
            bh_track  = [1.0]
            step_log  = []

            # TRADE LEDGER (window-level)
            trades = []
            open_trade = None
            prev_pos = 0.0
            eps = 1e-9

            START_EQUITY = 100_000.0

            # Track per-trade accumulated costs (from info0["trade_cost"])
            # Note: trade_cost is per-step cost (turnover * (cost+slip)), so we accumulate it while trade is open.

            executed_trade_count = 0
            signal_trade_count   = 0
            signal_trade_count_dyn = 0  # diagnostic only

            DIAG_THR = 0.2  # diagnostic-only fixed threshold

            # IMPORTANT: for DummyVecEnv, HOLD action must be shape (n_env, action_dim)
            # but we are predicting from model, so action already matches.
            last_eval_i = 0

            for i in range(len(df_window) - 1):
                # model.predict returns action shaped for VecEnv
                action, _ = model.predict(obs, deterministic=True)

                mu, sigma = get_mu_sigma(model, obs)

                obs, rew0, done0, info0 = _unwrap_step(env.step(action))

                # ---- HARDEN: protect against float(None) / missing info keys ----
                def _safe_float(x, default=0.0):
                    try:
                        if x is None:
                            return float(default)
                        return float(x)
                    except Exception:
                        return float(default)

                info0 = info0 or {}
                # normalize the fields that later get cast to float
                info0["nav"]        = _safe_float(info0.get("nav", nav_track[-1]), nav_track[-1])
                info0["base_ret"]   = _safe_float(info0.get("base_ret", 0.0), 0.0)
                info0["pos"]        = _safe_float(info0.get("pos", 0.0), 0.0)
                info0["trade_cost"] = _safe_float(info0.get("trade_cost", 0.0), 0.0)

                nav_track.append(info0["nav"])
                bh_track.append(bh_track[-1] * (1.0 + info0["base_ret"]))

                a = float(np.array(action).squeeze())
                dt_val = df_window["Datetime"].iloc[i+1] if "Datetime" in df_window.columns else None
                px     = float(df_window["Close"].iloc[i+1]) if "Close" in df_window.columns else np.nan

                pos_now = info0["pos"]
                trade_cost_step = info0["trade_cost"]
                nav_now = info0["nav"]

                def _close_trade(reason: str, exit_idx: int, exit_dt, exit_px: float, exit_nav: float):
                    nonlocal open_trade, trades

                    if open_trade is None:
                        return

                    entry_px = float(open_trade["EntryPrice"])
                    entry_pos = float(open_trade["EntryPos"])
                    side = open_trade["Side"]  # "LONG" or "SHORT"

                    # Direction: long = +1, short = -1
                    dir_sign = 1.0 if side == "LONG" else -1.0

                    # Price return (gross), scaled by position magnitude (continuous exposure)
                    gross_ret = dir_sign * abs(entry_pos) * ((exit_px / max(entry_px, 1e-9)) - 1.0)

                    # Net ret subtracts accumulated costs (already in "fraction" terms)
                    costs = float(open_trade["Costs"])
                    net_ret = gross_ret - costs

                    pnl_usd = (float(open_trade["ExitNAV"]) - float(open_trade["EntryNAV"])) * START_EQUITY

                    open_trade.update({
                        "ExitIndex": int(exit_idx),
                        "ExitDatetime": exit_dt,
                        "ExitPrice": float(exit_px),
                        "ExitNAV": float(exit_nav),
                        "BarsHeld": int(exit_idx - int(open_trade["EntryIndex"])),
                        "GrossReturnPct": float(gross_ret * 100.0),
                        "CostsPct": float(costs * 100.0),
                        "NetReturnPct": float(net_ret * 100.0),
                        "PnL_Usd": float(pnl_usd),
                        "ExitReason": str(reason),
                    })

                    trades.append(open_trade)
                    open_trade = None

                # If trade is open, accumulate step cost
                if open_trade is not None:
                    open_trade["Costs"] += trade_cost_step

                if open_trade is not None:
                    entry_px = float(open_trade["EntryPrice"])
                    entry_pos = float(open_trade["EntryPos"])
                    side = open_trade["Side"]
                    dir_sign = 1.0 if side == "LONG" else -1.0

                    # trade return (gross) at current px
                    gross_ret_now = dir_sign * abs(entry_pos) * ((px / max(entry_px, 1e-9)) - 1.0)

                    # update MFE/MAE (in return terms, not USD)
                    open_trade["MFE"] = max(float(open_trade.get("MFE", gross_ret_now)), gross_ret_now)
                    open_trade["MAE"] = min(float(open_trade.get("MAE", gross_ret_now)), gross_ret_now)

                # --- Detect transitions ---
                was_flat = abs(prev_pos) < eps
                now_flat = abs(pos_now) < eps
                sign_flip = (abs(prev_pos) >= eps) and (abs(pos_now) >= eps) and (np.sign(prev_pos) != np.sign(pos_now))

                # EXIT if position went nonzero -> 0
                if (not was_flat) and now_flat:
                    _close_trade(
                        reason=("TP/SL" if bool(info0.get("forced_close", False)) else
                                "SLO" if bool(info0.get("slo_triggered", False)) else
                                "FLAT"),
                        exit_idx=(i + 1),
                        exit_dt=dt_val,
                        exit_px=px,
                        exit_nav=nav_now
                    )

                # EXIT + RE-ENTER if sign flip
                elif sign_flip:
                    # Close previous using current bar as exit
                    _close_trade(
                        reason="SIGN_FLIP",
                        exit_idx=(i + 1),
                        exit_dt=dt_val,
                        exit_px=px,
                        exit_nav=nav_now,
                        "MFE_Pct": float(open_trade.get("MFE", 0.0) * 100.0),
                        "MAE_Pct": float(open_trade.get("MAE", 0.0) * 100.0),
                    )

                    # Open new trade immediately at same bar
                    side = "LONG" if pos_now > 0 else "SHORT"
                    open_trade = {
                        "Ticker": ticker,
                        "Window": f"{start}-{end}",
                        "WindowIdx": int(w_idx + 1),
                        "Prefix": prefix,
                        "Side": side,
                        "EntryIndex": int(i + 1),
                        "EntryDatetime": dt_val,
                        "EntryPrice": float(px),
                        "EntryPos": float(pos_now),
                        "EntryNAV": float(nav_now),
                        "Costs": 0.0,  # start counting costs after entry
                        "MFE": 0.0,
                        "MAE": 0.0,
                        "ExitIndex": None,
                        "ExitDatetime": None,
                        "ExitPrice": None,
                        "ExitNAV": None,
                        "BarsHeld": None,
                        "GrossReturnPct": None,
                        "CostsPct": None,
                        "NetReturnPct": None,
                        "PnL_Usd": None,
                        "ExitReason": None,
                    }

                # ENTRY if flat -> nonzero
                elif was_flat and (not now_flat):
                    side = "LONG" if pos_now > 0 else "SHORT"
                    open_trade = {
                        "Ticker": ticker,
                        "Window": f"{start}-{end}",
                        "WindowIdx": int(w_idx + 1),
                        "Prefix": prefix,
                        "Side": side,
                        "EntryIndex": int(i + 1),
                        "EntryDatetime": dt_val,
                        "EntryPrice": float(px),
                        "EntryPos": float(pos_now),
                        "EntryNAV": float(nav_now),
                        "Costs": 0.0,
                        "MFE": 0.0,
                        "MAE": 0.0,
                        "ExitIndex": None,
                        "ExitDatetime": None,
                        "ExitPrice": None,
                        "ExitNAV": None,
                        "BarsHeld": None,
                        "GrossReturnPct": None,
                        "CostsPct": None,
                        "NetReturnPct": None,
                        "PnL_Usd": None,
                        "ExitReason": None,
                    }

                # Update prev_pos for next iteration
                prev_pos = pos_now

                # “signal” trades (fixed threshold) — diagnostic only
                if (a > DIAG_THR) or (a < -DIAG_THR):
                    signal_trade_count += 1

                # real trades executed by env friction logic
                if bool(info0.get("executed_trade", False)):
                    executed_trade_count += 1

                # next-bar return (diagnostic label)
                if i + 2 < len(df_window):
                    p0 = float(df_window["Close"].iloc[i+1])
                    p1 = float(df_window["Close"].iloc[i+2])
                    next_ret = 0.0 if p0 <= 0 else (p1 - p0) / p0
                else:
                    next_ret = 0.0

                step_log.append({
                    "Index": i+1,
                    "Datetime": dt_val,
                    "Close": px,
                    "Action": a,
                    "mu": mu,
                    "sigma": sigma,
                    "nav": nav_track[-1],
                    "ret_t": float(info0.get("ret_t", 0.0)),
                    "next_ret": float(next_ret),
                    "reward": float(rew0),
                    "pos": float(info0.get("pos", 0.0)),
                    "trade_cost": float(info0.get("trade_cost", 0.0)),
                    "base_ret": float(info0.get("base_ret", 0.0)),
                    "rel_alpha": float(info0.get("rel_alpha", 0.0)),
                    "mom": float(info0.get("mom", 0.0)),
                    "vol_t": float(info0.get("vol_t", 0.0)),
                    "vol_ref": float(info0.get("vol_ref", 0.0)),
                    "vol_gate": float(info0.get("vol_gate", 0.0)),
                    "vol_scale": float(info0.get("vol_scale", 1.0)),
                    "vol_gated": bool(info0.get("vol_gated", False)),
                    "tp_pct": float(info0.get("tp_pct", 0.0)),
                    "sl_pct": float(info0.get("sl_pct", 0.0)),
                    "forced_close": bool(info0.get("forced_close", False)),
                    "slo_triggered": bool(info0.get("slo_triggered", False)),
                    "slo_locked": bool(info0.get("slo_locked", False)),
                    "slo_lock_until": int(info0.get("slo_lock_until", -1)),
                    "drawdown": float(info0.get("drawdown", 0.0)),
                })

                last_eval_i = i + 1
                if done0:
                    break

              step_df = pd.DataFrame(step_log)

            avg_abs_exposure = float(step_df["pos"].abs().mean()) if "pos" in step_df.columns and len(step_df) else 0.0

            turnover = (
                float(step_df["turnover_step"].sum())
                if "turnover_step" in step_df.columns and len(step_df)
                else float(step_df["delta_pos"].abs().sum())
                if "delta_pos" in step_df.columns and len(step_df)
                else 0.0
            )

            # holding time from trade ledger (bars held)
            if len(trades) > 0:
                tdf = pd.DataFrame(trades)
                avg_bars_held = float(pd.to_numeric(tdf["BarsHeld"], errors="coerce").dropna().mean()) if "BarsHeld" in tdf.columns else 0.0
            else:
                avg_bars_held = 0.0

            # Classification stats (using thr)
            abs_actions = np.array([abs(float(r["Action"])) for r in step_log], dtype=float)

            # Classification stats (using thr)
            if len(abs_actions) > 0:
                thr_window = float(np.quantile(abs_actions, 0.70))
                thr_window = float(np.clip(thr_window, 0.08, 0.30))
            else:
                thr_window = 0.2

            correct = total = 0
            tp_buy = fp_buy = tp_sell = fp_sell = 0

            for r in step_log:
                a = float(r["Action"])
                ret_t = float(r.get("next_ret", 0.0))

                if a > thr_window:
                    sig = "BUY"
                elif a < -thr_window:
                    sig = "SELL"
                else:
                    sig = "HOLD"

                if sig == "BUY":
                    total += 1
                    if ret_t > 0:
                        tp_buy += 1
                        correct += 1
                    else:
                        fp_buy += 1

                elif sig == "SELL":
                    total += 1
                    if ret_t < 0:
                        tp_sell += 1
                        correct += 1
                    else:
                        fp_sell += 1

            precision_long   = tp_buy  / (tp_buy  + fp_buy  + 1e-9)
            precision_short  = tp_sell / (tp_sell + fp_sell + 1e-9)
            precision_trades = (tp_buy + tp_sell) / ((tp_buy + tp_sell) + (fp_buy + fp_sell) + 1e-9)
            step_accuracy    = round(correct / total, 4) if total > 0 else 0.0

            # Keep this as REAL executed trades (cooldown/min_trade_delta)
            trade_count = int(executed_trade_count)

            # Dynamic signal trade count (post-hoc diagnostic)
            signal_trade_count_dyn = int(np.sum(abs_actions > thr_window)) if len(abs_actions) > 0 else 0

            # Metrics (use thr_window below)
            final_value = float(nav_track[-1]) * 100_000.0
            hold_value  = float(bh_track[-1])  * 100_000.0

            returns = pd.Series(nav_track).pct_change().fillna(0.0)
            sharpe  = float((returns.mean() / (returns.std() + 1e-9)) * _annualization_factor(df_window))
            drawdown = float(
                ((pd.Series(nav_track).cummax() - pd.Series(nav_track)) /
                pd.Series(nav_track).cummax()).max() * 100.0
            )

            # TRADE LEDGER finalization
            # If the episode ended while a trade is still open, close at last known bar
            if open_trade is not None:
                last_idx = int(last_eval_i)  # <- MUST be tracked in loop
                last_dt = df_window["Datetime"].iloc[last_idx] if "Datetime" in df_window.columns else None
                last_px = float(df_window["Close"].iloc[last_idx]) if "Close" in df_window.columns else np.nan
                last_nav = float(nav_track[-1])

                entry_px  = float(open_trade["EntryPrice"])
                entry_pos = float(open_trade["EntryPos"])
                side = open_trade["Side"]
                dir_sign = 1.0 if side == "LONG" else -1.0

                gross_ret = dir_sign * abs(entry_pos) * ((last_px / max(entry_px, 1e-9)) - 1.0)
                costs = float(open_trade["Costs"])
                net_ret = gross_ret - costs

                # NAV-consistent PnL in USD
                pnl_usd = (float(last_nav) - float(open_trade["EntryNAV"])) * START_EQUITY

                open_trade.update({
                    "ExitIndex": int(last_idx),
                    "ExitDatetime": last_dt,
                    "ExitPrice": float(last_px),
                    "ExitNAV": float(last_nav),
                    "BarsHeld": int(last_idx - int(open_trade["EntryIndex"])),
                    "GrossReturnPct": float(gross_ret * 100.0),
                    "CostsPct": float(costs * 100.0),
                    "NetReturnPct": float(net_ret * 100.0),
                    "PnL_Usd": float(pnl_usd),
                    "ExitReason": "EOD",
                })
                trades.append(open_trade)
                open_trade = None

            trades_path = os.path.join(RUN_RESULTS_DIR, f"{prefix}_trades.csv")
            pd.DataFrame(trades).to_csv(trades_path, index=False)
            logging.info(f"[TRADES] Saved trade ledger → {trades_path} | trades={len(trades)}")

            # Trade_count reflect REAL executed trades (cooldown/min_trade_delta)
            executed_trade_count = int(executed_trade_count)
            signal_trade_count_fixed = int(signal_trade_count)

            # Save model + VecNormalize (core artifacts first)
            model.save(model_path)

            save_ok = False
            save_err = None

            for candidate in (env, getattr(env, "venv", None)):
                if candidate is None:
                    continue
                if hasattr(candidate, "save"):
                    try:
                        candidate.save(vecnorm_path)
                        save_ok = True
                        break
                    except Exception as e:
                        save_err = e

            if not save_ok:
                logging.warning(
                    f"Could not save VecNormalize for {ticker} {start}-{end}: {save_err}"
                )
                try:
                    if os.path.exists(model_path):
                        os.remove(model_path)
                except Exception:
                    pass
                raise RuntimeError(f"VecNormalize save failed: {save_err}")

            # feature list used for this window
            feature_cols = [c for c in df_window.columns if c not in ("Datetime", "Symbol")]
            feature_cols = [c for c in feature_cols if pd.api.types.is_numeric_dtype(df_window[c])]

            model_info = {
                "model": "PPO",
                "ticker": ticker,
                "window": f"{start}-{end}",
                "window_idx": int(w_idx + 1),
                "prefix": prefix,
                "date_trained_utc": now_utc().strftime("%Y-%m-%dT%H:%M:%SZ"),
                "framework": "stable-baselines3",
                "device": DEVICE_STR,
                "seed": SEED,
                "timesteps": int(timesteps),
                "env_kwargs": dict(ENV_KWARGS),
                "ppo_overrides": dict(ppo_overrides),
                "metrics": {
                    "final_portfolio": round(final_value, 2),
                    "buy_hold": round(hold_value, 2),
                    "sharpe": round(sharpe, 6),
                    "drawdown_pct": round(drawdown, 6),
                    "trade_count": int(executed_trade_count),
                    "action_threshold": float(thr_window),
                    "accuracy": float(step_accuracy),
                    "precision_long": float(precision_long),
                    "precision_short": float(precision_short),
                    "precision_trades": float(precision_trades),
                    "avg_abs_exposure": round(avg_abs_exposure, 6),
                    "turnover": round(turnover, 6),
                    "avg_bars_held": round(avg_bars_held, 3),
                }
            }

            write_master_bundle(
                prefix=prefix,
                features=feature_cols,
                threshold=float(thr_window),
                model_info=model_info
            )

            # ---- Save detailed predictions (must happen BEFORE manifest_record_paths) ----
            pred_path = os.path.join(RUN_RESULTS_DIR, f"{prefix}_predictions.csv")
            pd.DataFrame(step_log).to_csv(pred_path, index=False)
            logging.info(f"Saved predictions to {pred_path}")

            # ---- Save compat predictions (SINGLE COPY; no duplicates) ----
            compat_rows = []
            for r in step_log:
                a = float(r["Action"])
                if a > thr_window:
                    signal = "BUY"
                elif a < -thr_window:
                    signal = "SELL"
                else:
                    signal = "HOLD"

                compat_rows.append({
                    "Index": r["Index"],
                    "Datetime": r["Datetime"],
                    "Close": r["Close"],
                    "Action": a,
                    "Signal": signal,
                    "PortfolioValue": r["nav"],
                    "Reward": r.get("reward", np.nan),
                })

            compat_path = os.path.join(RUN_RESULTS_DIR, f"{prefix}_predictions_compat.csv")
            pd.DataFrame(compat_rows).to_csv(compat_path, index=False)
            logging.info(f"Saved compatibility predictions to {compat_path}")

            manifest_record_paths(
                prefix,
                {
                    "model_zip": model_path,
                    "vecnorm_pkl": vecnorm_path,
                    "predictions_csv": pred_path,
                    "predictions_compat_csv": compat_path,
                    "trades_csv": trades_path,
                },
                do_hashes=ENABLE_HASHES
            )

            # status last
            manifest_update(prefix, {
                "status": "trained",
                "ticker": ticker,
                "window": f"{start}-{end}",
                "window_idx": int(w_idx + 1),
                "sharpe": float(sharpe),
                "updated_at": now_utc().isoformat(),
            })

            # Summary row
            result_row = {
                "Ticker": ticker,
                "Window": f"{start}-{end}",
                "WindowIdx": int(w_idx + 1),
                "Prefix": prefix,
                "PPO_Portfolio": round(final_value, 2),
                "BuyHold": round(hold_value, 2),
                "Sharpe": round(sharpe, 3),
                "Drawdown_%": round(drawdown, 2),
                "Winner": "PPO" if final_value > hold_value else "Buy & Hold",
                "Action_Threshold": round(thr_window, 4),
                "Accuracy": step_accuracy,
                "Trade_Count": int(executed_trade_count),
                "Signal_Trade_Count": int(signal_trade_count),
                "Signal_Trade_Count_Dyn": int(signal_trade_count_dyn),
                "Executed_Trade_Count": int(executed_trade_count),
                "Precision_Long": round(precision_long, 4),
                "Precision_Short": round(precision_short, 4),
                "Precision_Trades": round(precision_trades, 4),
            }

            results.append(result_row)

            meta = {
                "result": result_row,
                "features": feature_cols,
                "prefix": prefix,
                "model_path": model_path,
                "vecnorm_path": vecnorm_path,
            }

            item = (result_row["Sharpe"], prefix, meta)
            if len(top_heap) < TOP_N_WINDOWS:
                heapq.heappush(top_heap, item)
            else:
                if item[0] > top_heap[0][0]:
                    heapq.heapreplace(top_heap, item)

            logging.info(
                f"{ticker} | Window {w_idx+1} runtime: "
                f"{round(time.time() - window_start_time, 2)}s"
            )
        finally:
            try:
                env.close()
            except Exception:
                pass
            del env
            try:
                del model
            except Exception:
                pass
            gc.collect()
            try:
                torch.cuda.empty_cache()
            except Exception:
                pass

    if skipped_windows:
        logging.info(
            f"{ticker} skipped windows (already complete): {', '.join(skipped_windows)}"
        )
        record_skips_global(
            ticker,
            skipped_windows=skipped_windows,
            total_windows=len(windows),
            fully_skipped=False,
        )

    # Save top-N QC-compatible
    top_list = sorted(top_heap, key=lambda t: t[0], reverse=True)
    for _, _, meta in top_list:
        artifact_for_save = {
            "model": None,  # we're copying from disk, not re-saving an in-memory model
            "model_path": meta["model_path"],
            "vecnorm_path": meta["vecnorm_path"],
            "features": meta["features"],
            "result": meta["result"],
            "prefix": meta["prefix"],
        }
        save_quantconnect_model(artifact_for_save, meta["prefix"], QC_TOP_DIR)

    return results

def process_ticker(ticker):
    try:
        hp = pick_params(ticker)
        return walkforward_ppo(
            df[df["Symbol"] == ticker].copy(),
            ticker,
            window_size=WINDOW_SIZE,
            step_size=STEP_SIZE,
            timesteps=TIMESTEPS,
            learning_rate=hp["lr"],
            ppo_overrides=hp,
        )
    except Exception as e:
        logging.error(f"{ticker}: training failed with {e}")
        return []


from concurrent.futures import ThreadPoolExecutor

def run_parallel_tickers(tickers,
                         out_path=os.path.join(RUN_RESULTS_DIR, "summary.csv"),
                         max_workers=None):
    if max_workers is None:
        max_workers = MAX_WORKERS

    logging.info(f"[PARALLEL] Using max_workers={max_workers} (MAX_WORKERS={MAX_WORKERS})")

    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        for res in ex.map(process_ticker, tickers):
            if res:
                results.extend(res)

    if results:
        pd.DataFrame(results).to_csv(out_path, index=False)
        logging.info(f"Saved summary to {out_path}")
    else:
        logging.warning("No results produced; summary not written.")

    logging.info("All tickers processed.")
    return results

def build_ppo_selector():
    """Aggregate all summary*.csv across runs and build selector JSON."""
    summary_files = glob.glob(
        os.path.join(BASE_RESULTS_DIR, "ppo_walkforward_results_*", "summary*.csv")
    )
    all_summaries = []

    for p in summary_files:
        try:
            tmp = pd.read_csv(p)
            tmp["RunFolder"] = os.path.dirname(p)
            all_summaries.append(tmp)
        except Exception as e:
            print(f"⚠️ Skipping {p} due to error: {e}")

    if not all_summaries:
        logging.warning("No PPO summaries found across walkforward results folders.")
        return

    combo = pd.concat(all_summaries, ignore_index=True)

    # Robust Sharpe handling (avoid string sorting / bad rows from older runs)
    if "Sharpe" in combo.columns:
        combo["Sharpe"] = pd.to_numeric(combo["Sharpe"], errors="coerce")
        combo = combo.dropna(subset=["Sharpe"])
    else:
        logging.warning("No 'Sharpe' column found in combined summaries; selector may be empty.")

    # Ensure key columns exist for robust ratios
    if "BuyHold" not in combo.columns:
        combo["BuyHold"] = np.nan
    if "PPO_Portfolio" not in combo.columns:
        combo["PPO_Portfolio"] = np.nan

    # parse Window "start-end" to WindowStart
    def _parse_window_start(w):
        if pd.isna(w):
            return None
        if isinstance(w, (int, float)):
            return int(w)
        parts = str(w).split("-")
        try:
            return int(parts[0])
        except Exception:
            return None

    combo["WindowStart"] = combo["Window"].apply(_parse_window_start)

    def _parse_run_ts(run_folder: str):
        try:
            base = os.path.basename(str(run_folder))
            # split on last "_" occurrence after prefix
            # base example: "ppo_walkforward_results_20260114_1545"
            parts = base.replace("ppo_walkforward_results_", "").split("_")
            if len(parts) >= 2:
                dt_str = parts[0] + parts[1]  # YYYYMMDDHHMM
                return pd.to_datetime(dt_str, format="%Y%m%d%H%M", errors="coerce")
        except Exception:
            pass
        return pd.NaT

    combo["RunTS"] = combo["RunFolder"].apply(_parse_run_ts)
    combo = combo.sort_values(["Ticker", "WindowStart", "RunTS"]).reset_index(drop=True)

    # If summaries already contain WindowIdx from training, keep it.
    if "WindowIdx" not in combo.columns:
        combo["WindowIdx"] = combo.groupby("Ticker").cumcount() + 1
    else:
        combo["WindowIdx"] = pd.to_numeric(combo["WindowIdx"], errors="coerce")

    combo = combo.drop_duplicates(subset=["Ticker", "WindowStart"], keep="last")

    best_by_symbol = (
        combo
        .sort_values("Sharpe", ascending=False)
        .groupby("Ticker")
        .first()
        .reset_index()
    )

    # If Drawdown_% missing (older runs), create it so rename won't break
    if "Drawdown_%" not in best_by_symbol.columns:
        best_by_symbol["Drawdown_%"] = np.nan

    # Ensure precision cols exist
    for col in ["Precision_Long", "Precision_Short", "Precision_Trades"]:
        if col not in best_by_symbol.columns:
            best_by_symbol[col] = None  # or np.nan if you prefer

    # Rename columns so everything downstream uses consistent names
    best_by_symbol = best_by_symbol.rename(columns={
        "Drawdown_%": "Drawdown",
        "PPO_Portfolio": "Final_Portfolio",
    })

    # Ensure Accuracy / Trade_Count exist
    if "Accuracy" not in best_by_symbol.columns:
        best_by_symbol["Accuracy"] = 0.0
    if "Trade_Count" not in best_by_symbol.columns:
        best_by_symbol["Trade_Count"] = None

    best_by_symbol["Model"] = MODEL_NAME

    # PPO vs Buy & Hold ratio (safe division)
    best_by_symbol["Rel_vs_BH"] = best_by_symbol.apply(
        lambda r: (r["Final_Portfolio"] / r["BuyHold"])
        if (pd.notna(r["BuyHold"]) and r["BuyHold"] not in (0, 0.0)) else np.nan,
        axis=1
    )

    # Save flat CSV for debugging
    best_by_symbol.to_csv(SELECTOR_FULL_PATH, index=False)
    print(f"Aggregated PPO selector saved to → {SELECTOR_FULL_PATH}")

    # Safety filters (tune as needed)
    df_sel = best_by_symbol.copy()
    gates = (
        (df_sel["Sharpe"].fillna(-999) > 0.0) &
        (df_sel["Drawdown"].fillna(999) < 50.0) &
        (df_sel["Final_Portfolio"].fillna(0) > 80_000) &
        (df_sel["Rel_vs_BH"].fillna(0) >= 0.95)   # PPO ≥ 95% of B&H; change to >1.0 to enforce beat
    )
    df_sel = df_sel[gates].copy()

    df_sel["prefix"] = (
        "ppo_"
        + df_sel["Ticker"].astype(str)
        + "_window"
        + df_sel["WindowIdx"].astype(int).astype(str)
    )

    df_sel["artifact_path"] = df_sel["prefix"].apply(
        lambda p: os.path.join(FINAL_MODEL_DIR, f"{p}_model.zip")
    )
    df_sel["vecnorm_path"] = df_sel["prefix"].apply(
        lambda p: os.path.join(FINAL_MODEL_DIR, f"{p}_vecnorm.pkl")
    )

    EPS = 0.03  # 3% of top-sharpe for "close enough"
    selected_models = {}

    def safe_int(v, default=0):
        if v is None:
            return int(default)
        try:
            import math
            if isinstance(v, float) and math.isnan(v):
                return int(default)
        except TypeError:
            pass
        try:
            return int(v)
        except (ValueError, TypeError):
            return int(default)

    def safe_float(v, default=0.0):
        if v is None:
            return float(default)
        try:
            import math
            if isinstance(v, float) and math.isnan(v):
                return float(default)
        except TypeError:
            pass
        try:
            return float(v)
        except (ValueError, TypeError):
            return float(default)

    for ticker, group in df_sel.groupby("Ticker"):
        group_sorted = group.sort_values("Sharpe", ascending=False)
        top = group_sorted.iloc[0]
        second = group_sorted.iloc[1] if len(group_sorted) > 1 else None

        if (second is not None) and (
            abs(top["Sharpe"] - second["Sharpe"]) <= abs(top["Sharpe"]) * EPS
        ):
            mode = "ensemble"
            primary, secondary = top["Model"], second["Model"]
        else:
            mode = "single"
            primary, secondary = top["Model"], None

        selected_models[ticker] = {
            "model": MODEL_NAME,
            "score": round(safe_float(top["Sharpe"]), 4),
            "return": round(safe_float(top["Final_Portfolio"]), 2),
            "sharpe": round(safe_float(top["Sharpe"]), 3),
            "drawdown": round(safe_float(top["Drawdown"]), 2),
            "sortino": None,
            "turnover": None,
            "trade_count": safe_int(top.get("Trade_Count", 0)),
            "precision": {
                "long":   safe_float(top.get("Precision_Long", 0.0)),
                "short":  safe_float(top.get("Precision_Short", 0.0)),
                "trades": safe_float(top.get("Precision_Trades", 0.0)),
            },
            "stability": {},
            "regime": "unknown",
            "rl_profile": "fast",
            "artifact": {
                "path": top["artifact_path"],
                "vecnorm": top["vecnorm_path"],
                "features": None,
                "load_ms": 180,
                "mem_mb": 512,
                "exists": os.path.exists(top["artifact_path"]),
            },
            "selection": {
                "mode": mode,
                "primary": primary,
                "secondary": secondary,
            },
        }

    with open(SELECTOR_JSON_PATH, "w") as f:
        json.dump(selected_models, f, indent=2)

    print(f"Final enhanced PPO selector JSON saved to → {SELECTOR_JSON_PATH}")

if __name__ == "__main__":
    hc = health_check_latest_run(BASE_RESULTS_DIR)
    if hc.get("ok"):
        logging.info(f"[HEALTH] Latest run folder: {hc['latest_run_dir']}")
        logging.info(f"[HEALTH] Summary files: {hc['summary_files']}")
        logging.info(f"[HEALTH] Rows total: {hc['rows_total']} | tickers: {len(hc['tickers'])}")
    else:
        logging.warning(f"[HEALTH] {hc.get('reason')}")
    logging.info(f"RUN_RESULTS_DIR   = {RUN_RESULTS_DIR}")
    logging.info(f"FINAL_MODEL_DIR  = {FINAL_MODEL_DIR}")
    logging.info(f"BASE_RESULTS_DIR = {BASE_RESULTS_DIR}")

    min_rows = WINDOW_SIZE + 50  # small buffer so we have at least one window
    all_symbols = df["Symbol"].value_counts()
    candidate_symbols = []

    for sym, n in all_symbols.items():
        if n >= min_rows:
            candidate_symbols.append(sym)
        else:
            logging.warning(f"Skipping {sym}: only {n} rows (< {min_rows} required)")

    if not candidate_symbols:
        logging.error("No symbols have enough rows for the current WINDOW_SIZE. Nothing to train.")
    else:
        logging.info(f"Training candidate symbols: {candidate_symbols}")

    needed_cols = ["Close", "Datetime"]
    if ENABLE_WAVELET:
        needed_cols.append("Denoised_Close")
    if ENABLE_SENTIMENT:
        needed_cols.append("SentimentScore")

    valid_symbols = []
    for sym in candidate_symbols:
        cols = set(df.loc[df["Symbol"] == sym].columns)
        missing = [c for c in needed_cols if c not in cols]
        if missing:
            logging.warning(f"Skipping {sym}: missing required cols {missing}")
        else:
            valid_symbols.append(sym)

    if not valid_symbols:
        logging.error("No symbols passed the feature/column checks. Nothing to train.")
    else:
        logging.info(f"Final training universe: {valid_symbols}")

    all_results = []

    if test_mode:
        # Optional: shrink timesteps and/or window size in test mode
        TIMESTEPS = 10_000   # lighter test
        WINDOW_SIZE = 1200  # uncomment if you want faster test runs
        STEP_SIZE   = 600

        test_stocks = ["AAPL", "NVDA", "MSFT"]
        present = [s for s in test_stocks if s in valid_symbols]
        if not present:
            logging.warning("Test mode: none of ['AAPL','NVDA','MSFT'] present after filters.")
        else:
            logging.info(f"Test mode: running on {present}")

        for sym in present:
            logging.info(f">>> [TEST_MODE] Processing {sym}")
            res = process_ticker(sym)
            logging.info(f"{sym}: produced {len(res)} window summaries")
            if res:
                all_results.extend(res)

        summary_path = os.path.join(RUN_RESULTS_DIR, "summary_test_mode.csv")
        if all_results:
            pd.DataFrame(all_results).to_csv(summary_path, index=False)
            logging.info(f"Test-mode summary saved to {summary_path}")
        else:
            logging.warning("Test mode finished but no results were generated (no windows, or all skipped).")

    else:
        logging.info(f"Starting full parallel PPO walkforward run... (MAX_WORKERS={MAX_WORKERS})")
        summary_results = run_parallel_tickers(valid_symbols, max_workers=MAX_WORKERS)

        if not summary_results:
            logging.warning("No results generated in full run (check logs for skips/length issues).")
        else:
            summary_path = os.path.join(RUN_RESULTS_DIR, "summary.csv")
            pd.DataFrame(summary_results).to_csv(summary_path, index=False)
            logging.info(f"Summary saved to {summary_path}")

    try:
        build_ppo_selector()
    except Exception as e:
        logging.error(f"build_ppo_selector failed: {e}")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/tmp/ipython-input-754822098.py:215: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_at": datetime.utcnow().isoformat(),
2026-02-16 02:19:13,545 - INFO - [CONFIG] MAX_WORKERS=1 | cuda_avail=True | test_mode=True | USE_GPU=False | DEVICE_STR=cpu
2026-02-16 02:19:13,603 - INFO - [CONFIG] Wrote run config → /content/drive/MyDrive/Results_May_2025/ppo_walkforward_results_20260216_0219/config.json
2026-02-16 02:19:14,372 - WARNING - [HEALTH] Latest run has no readable sum

Aggregated PPO selector saved to → /content/drive/MyDrive/Results_May_2025/ppo_model_selector_FULL.csv
Final enhanced PPO selector JSON saved to → /content/drive/MyDrive/Results_May_2025/ppo_model_selector_final.json


In [ ]:
import os, glob, re, json
import pandas as pd
from datetime import datetime

BASE = "/content/drive/MyDrive/Results_May_2025"
MASTER_DIR = os.path.join(BASE, "ppo_models_master")
QC_DIR     = os.path.join(BASE, "ppo_models_QC_TOP")

def _count(d):
    return {
        "zip": len(glob.glob(os.path.join(d,"*.zip"))),
        "pkl": len(glob.glob(os.path.join(d,"*.pkl"))),
        "features": len(glob.glob(os.path.join(d,"*_features.json"))),
        "prob_cfg": len(glob.glob(os.path.join(d,"*_probability_config.json"))),
        "info": len(glob.glob(os.path.join(d,"*_model_info.json"))),
    }

def _prefixes_from_models(d):
    zips = glob.glob(os.path.join(d, "ppo_*_model.zip"))
    prefs = sorted(set(re.sub(r"_model\.zip$", "", os.path.basename(p)) for p in zips))
    return prefs

def _latest_run_folder():
    runs = sorted(glob.glob(os.path.join(BASE, "ppo_walkforward_results_*")))
    return runs[-1] if runs else None

def _latest_summary_in_run(run_folder):
    if not run_folder:
        return None
    candidates = sorted(glob.glob(os.path.join(run_folder, "summary*.csv")))
    return candidates[-1] if candidates else None

def _check_bundle(prefix, folder, extended=False):
    core = ["_model.zip","_vecnorm.pkl","_features.json","_probability_config.json","_model_info.json"]
    extra = ["_predictions.csv","_predictions_compat.csv","_trades.csv"]

    need = core + (extra if extended else [])
    return {s: os.path.exists(os.path.join(folder, prefix+s)) for s in need}

print("\n=== PPO ARTIFACT HEALTH CHECK ===")
print("Time:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("BASE:", BASE)
print("MASTER_DIR:", MASTER_DIR, "| exists:", os.path.exists(MASTER_DIR))
print("QC_DIR:", QC_DIR, "| exists:", os.path.exists(QC_DIR))

latest_run = _latest_run_folder()
latest_summary = _latest_summary_in_run(latest_run)
print("\n[Run Output]")
print("Latest run folder:", latest_run)
print("Latest summary file:", latest_summary)

latest_manifest = _latest_manifest_in_run(latest_run)
print("Latest manifest file:", latest_manifest)

if latest_manifest:
    try:
        with open(latest_manifest, "r") as f:
            man = json.load(f)
        art = man.get("artifacts", {})
        print("Manifest artifact prefixes:", len(art))
        # show a few keys so you can confirm it’s populated
        print("Manifest sample prefixes:", list(art.keys())[:10])
    except Exception as e:
        print("Could not read manifest:", e)

if latest_summary and os.path.exists(latest_summary):
    try:
        s = pd.read_csv(latest_summary)
        print("Summary rows:", len(s))
        print("Tickers in summary:", sorted(s["Ticker"].dropna().astype(str).unique().tolist())[:50])
    except Exception as e:
        print("Could not read summary:", e)

print("\n[Counts]")
print("MASTER counts:", _count(MASTER_DIR) if os.path.exists(MASTER_DIR) else "missing folder")
print("QC_TOP counts:", _count(QC_DIR) if os.path.exists(QC_DIR) else "missing folder")

master_prefixes = _prefixes_from_models(MASTER_DIR) if os.path.exists(MASTER_DIR) else []
qc_prefixes     = _prefixes_from_models(QC_DIR)     if os.path.exists(QC_DIR) else []

print("\n[Prefixes]")
print("MASTER prefixes:", master_prefixes[:15], ("... total "+str(len(master_prefixes)) if len(master_prefixes)>15 else ""))
print("QC_TOP prefixes:", qc_prefixes[:15], ("... total "+str(len(qc_prefixes)) if len(qc_prefixes)>15 else ""))

print("\n[QC_TOP Bundle Integrity — should be all True]")
for pref in qc_prefixes[:10]:  # show first 10
    status = _check_bundle(pref, QC_DIR, extended=True)
    ok = all(status.values())
    print(pref, "OK" if ok else "MISSING", status)

def check_any(prefix):
    print("\n[Check prefix]", prefix)
    if os.path.exists(os.path.join(QC_DIR, prefix+"_model.zip")):
        folder = QC_DIR
        where = "QC_TOP"
    elif os.path.exists(os.path.join(MASTER_DIR, prefix+"_model.zip")):
        folder = MASTER_DIR
        where = "MASTER"
    else:
        print("Not found in MASTER or QC_TOP.")
        return
    status = _check_bundle(prefix, folder)
    print("Found in:", where, "->", folder)
    print(status)




=== PPO ARTIFACT HEALTH CHECK ===
Time: 2026-02-16 02:20:15
BASE: /content/drive/MyDrive/Results_May_2025
MASTER_DIR: /content/drive/MyDrive/Results_May_2025/ppo_models_master | exists: True
QC_DIR: /content/drive/MyDrive/Results_May_2025/ppo_models_QC_TOP | exists: True

[Run Output]
Latest run folder: /content/drive/MyDrive/Results_May_2025/ppo_walkforward_results_20260216_0219
Latest summary file: /content/drive/MyDrive/Results_May_2025/ppo_walkforward_results_20260216_0219/summary_test_mode.csv
Latest manifest file: /content/drive/MyDrive/Results_May_2025/ppo_walkforward_results_20260216_0219/artifact_manifest.json
Manifest artifact prefixes: 2
Manifest sample prefixes: ['ppo_NVDA_window1', 'ppo_MSFT_window1']
Summary rows: 2
Tickers in summary: ['MSFT', 'NVDA']

[Counts]
MASTER counts: {'zip': 2, 'pkl': 2, 'features': 2, 'prob_cfg': 2, 'info': 2}
QC_TOP counts: {'zip': 2, 'pkl': 2, 'features': 2, 'prob_cfg': 2, 'info': 2}

[Prefixes]
MASTER prefixes: ['ppo_MSFT_window1', 'ppo_NVD

In [ ]:
import os, glob, re

BASE = "/content/drive/MyDrive/Results_May_2025"
MASTER = os.path.join(BASE, "ppo_models_master")
QC = os.path.join(BASE, "ppo_models_QC_TOP")

need = ["_model.zip","_vecnorm.pkl","_features.json","_probability_config.json","_model_info.json"]

def audit(folder):
    zips = glob.glob(os.path.join(folder, "ppo_*_model.zip"))
    prefs = sorted(set(re.sub(r"_model\.zip$", "", os.path.basename(p)) for p in zips))
    bad = []
    for pref in prefs:
        missing = [s for s in need if not os.path.exists(os.path.join(folder, pref+s))]
        if missing:
            bad.append((pref, missing))
    return prefs, bad

for name, folder in [("MASTER", MASTER), ("QC_TOP", QC)]:
    prefs, bad = audit(folder)
    print(name, "prefixes:", len(prefs), "| incomplete:", len(bad))
    if bad:
        print("Example missing:", bad[0])


MASTER prefixes: 2 | incomplete: 0
QC_TOP prefixes: 2 | incomplete: 0
